# [Essential basic functionality](https://pandas.pydata.org/pandas-docs/stable/user_guide/basics.html)
Here we discuss a lot of the essential functionality common to the pandas data structures. To begin, let’s create some example objects like we did in the 10 minutes to pandas section:

In [1]:
import numpy as np
import pandas as pd

In [2]:
index = pd.date_range("1/1/2000", periods=8)
s = pd.Series(np.random.randn(5), index=["a", "b", "c", "d", "e"])
df = pd.DataFrame(np.random.randn(8, 3), index=index, columns=["A", "B", "C"])

## Head and tail
To view a small sample of a Series or DataFrame object, use the head() and tail() methods. The default number of elements to display is five, but you may pass a custom number.

In [3]:
long_series = pd.Series(np.random.randn(1000))
long_series.head()

0    0.402303
1   -1.956801
2   -0.817306
3   -1.504956
4   -1.250553
dtype: float64

In [4]:
long_series.tail(3)

997    1.013036
998    0.335010
999    1.507347
dtype: float64

## Attributes and underlying data
pandas objects have a number of attributes enabling you to access the metadata

* shape: gives the axis dimensions of the object, consistent with ndarray
* Axis labels
  * Series: index (only axis)
  * DataFrame: index (rows) and columns

> Note, these attributes can be safely assigned to!

In [5]:
df[:2]

,A,B,C
2000-01-01,-0.705073,0.85772,-0.217983
2000-01-02,0.427849,-0.93131,1.730415


In [6]:
df.columns = [x.lower() for x in df.columns]
print(df)

                   a         b         c
2000-01-01 -0.705073  0.857720 -0.217983
2000-01-02  0.427849 -0.931310  1.730415
2000-01-03  0.095698  0.298959  1.244970
2000-01-04 -0.827048  0.705629 -0.771791
2000-01-05 -1.528118 -1.737220  0.862065
2000-01-06 -0.047342 -1.249752 -1.539455
2000-01-07  1.428360 -0.573503  1.152012
2000-01-08  0.514042 -0.755157  0.624050


pandas objects (Index, Series, DataFrame) can be thought of as containers for arrays, which hold the actual data and do the actual computation. For many types, the underlying array is a numpy.ndarray. However, pandas and 3rd party libraries may extend NumPy’s type system to add support for custom arrays (see dtypes).

To get the actual data inside a Index or Series, use the .array property

In [7]:
s.array

<PandasArray>
[ 1.0813595456338783,  0.7358526379034812,  1.2612137410283988,
 0.00236966898212244, -0.4361931375934344]
Length: 5, dtype: float64

In [8]:
s.index.array

<PandasArray>
['a', 'b', 'c', 'd', 'e']
Length: 5, dtype: object

array will always be an ExtensionArray. The exact details of what an ExtensionArray is and why pandas uses them are a bit beyond the scope of this introduction. See dtypes for more.

If you know you need a NumPy array, use to_numpy() or numpy.asarray().

In [9]:
s.to_numpy()

array([ 1.08135955,  0.73585264,  1.26121374,  0.00236967, -0.43619314])

In [10]:
np.asarray(s)

array([ 1.08135955,  0.73585264,  1.26121374,  0.00236967, -0.43619314])

When the Series or Index is backed by an ExtensionArray, to_numpy() may involve copying data and coercing values. See dtypes for more.

to_numpy() gives some control over the dtype of the resulting numpy.ndarray. For example, consider datetimes with timezones. NumPy doesn’t have a dtype to represent timezone-aware datetimes, so there are two possibly useful representations:

1. An object-dtype numpy.ndarray with Timestamp objects, each with the correct tz
2. A datetime64[ns] -dtype numpy.ndarray, where the values have been converted to UTC and the timezone discarded

Timezones may be preserved with dtype=object

In [11]:
ser = pd.Series(pd.date_range("2000", periods=2, tz="CET"))
ser.to_numpy(dtype=object)

array([Timestamp('2000-01-01 00:00:00+0100', tz='CET', freq='D'),
       Timestamp('2000-01-02 00:00:00+0100', tz='CET', freq='D')],
      dtype=object)

Or thrown away with dtype='datetime64[ns]'

In [12]:
ser.to_numpy(dtype="datetime64[ns]")

array(['1999-12-31T23:00:00.000000000', '2000-01-01T23:00:00.000000000'],
      dtype='datetime64[ns]')

Getting the “raw data” inside a DataFrame is possibly a bit more complex. When your DataFrame only has a single data type for all the columns, DataFrame.to_numpy() will return the underlying data:

In [13]:
df.to_numpy()

array([[-0.70507344,  0.85772013, -0.21798308],
       [ 0.42784863, -0.93130977,  1.73041471],
       [ 0.09569783,  0.29895898,  1.24496956],
       [-0.82704809,  0.7056293 , -0.77179101],
       [-1.52811809, -1.7372202 ,  0.86206458],
       [-0.04734151, -1.24975244, -1.53945535],
       [ 1.42836018, -0.57350328,  1.15201206],
       [ 0.51404224, -0.75515723,  0.62404978]])

If a DataFrame contains homogeneously-typed data, the ndarray can actually be modified in-place, and the changes will be reflected in the data structure. For heterogeneous data (e.g. some of the DataFrame’s columns are not all the same dtype), this will not be the case. The values attribute itself, unlike the axis labels, cannot be assigned to.

>Note
>When working with heterogeneous data, the dtype of the resulting ndarray will be chosen to accommodate all of the data involved. For example, if strings are involved, the result will be of object dtype. If there are only floats and integers, the resulting array will be of float dtype.

In the past, pandas recommended Series.values or DataFrame.values for extracting the data from a Series or DataFrame. You’ll still find references to these in old code bases and online. Going forward, we recommend avoiding .values and using .array or .to_numpy(). .values has the following drawbacks:

1. When your Series contains an extension type, it’s unclear whether Series.values returns a NumPy array or the extension array. Series.array will always return an ExtensionArray, and will never copy data. Series.to_numpy() will always return a NumPy array, potentially at the cost of copying / coercing values.
2. When your DataFrame contains a mixture of data types, DataFrame.values may involve copying data and coercing values to a common dtype, a relatively expensive operation. DataFrame.to_numpy(), being a method, makes it clearer that the returned NumPy array may not be a view on the same data in the DataFrame.

## Accelerated operations
pandas has support for accelerating certain types of binary numerical and boolean operations using the numexpr library and the bottleneck libraries.

These libraries are especially useful when dealing with large data sets, and provide large speedups. numexpr uses smart chunking, caching, and multiple cores. bottleneck is a set of specialized cython routines that are especially fast when dealing with arrays that have nans.

Here is a sample (using 100 column x 100,000 row DataFrames):

| Operation | 0.11.0 (ms) | Prior Version (ms) | Ratio to Prior |
|-----------|-------------|--------------------|----------------|
| df1 > df2 | 13.32       | 125.35             | 0.1063         |
| df1 * df2 | 21.71       | 36.63              | 0.5928         |
| df1 + df2 | 22.04       | 36.50              | 0.6039         |

You are highly encouraged to install both libraries. See the section Recommended Dependencies for more installation info.

These are both enabled to be used by default, you can control this by setting the options:

In [14]:
pd.set_option("compute.use_bottleneck", False)
pd.set_option("compute.use_numexpr", False)

## Flexible binary operations
With binary operations between pandas data structures, there are two key points of interest:

Broadcasting behavior between higher- (e.g. DataFrame) and lower-dimensional (e.g. Series) objects.

Missing data in computations.

We will demonstrate how to manage these issues independently, though they can be handled simultaneously.

### Matching / broadcasting behavior
DataFrame has the methods add(), sub(), mul(), div() and related functions radd(), rsub(), … for carrying out binary operations. For broadcasting behavior, Series input is of primary interest. Using these functions, you can use to either match on the index or columns via the axis keyword:

In [15]:
df = pd.DataFrame(
    {
        "one": pd.Series(np.random.randn(3), index=["a", "b", "c"]),
        "two": pd.Series(np.random.randn(4), index=["a", "b", "c", "d"]),
        "three": pd.Series(np.random.randn(3), index=["b", "c", "d"]),
    }
)
print(df)

        one       two     three
a -0.005240  0.848099       NaN
b  1.323212 -0.196640  1.092488
c  0.380090 -0.416081  0.290022
d       NaN -0.788091 -1.715676


In [16]:
row = df.iloc[1]
column = df["two"]
df.sub(row, axis="columns")

,one,two,three
a,-1.328452,1.044739,NaN
b,0.000000,0.000000,0.000000
c,-0.943121,-0.219441,-0.802466
d,NaN,-0.591451,-2.808165


In [17]:
df.sub(row, axis=1)

,one,two,three
a,-1.328452,1.044739,NaN
b,0.000000,0.000000,0.000000
c,-0.943121,-0.219441,-0.802466
d,NaN,-0.591451,-2.808165


In [18]:
df.sub(column, axis="index")

,one,two,three
a,-0.853339,0.0,NaN
b,1.519851,0.0,1.289128
c,0.796171,0.0,0.706102
d,NaN,0.0,-0.927586


In [19]:
df.sub(column, axis=0)

,one,two,three
a,-0.853339,0.0,NaN
b,1.519851,0.0,1.289128
c,0.796171,0.0,0.706102
d,NaN,0.0,-0.927586


Furthermore you can align a level of a MultiIndexed DataFrame with a Series.

In [20]:
dfmi = df.copy()
dfmi.index = pd.MultiIndex.from_tuples(
    [(1, "a"), (1, "b"), (1, "c"), (2, "a")], names=["first", "second"]
)
dfmi.sub(column, axis=0, level="second")

one      two     three
first second                             
1     a      -0.853339  0.00000       NaN
      b       1.519851  0.00000  1.289128
      c       0.796171  0.00000  0.706102
2     a            NaN -1.63619 -2.563775

Series and Index also support the divmod() builtin. This function takes the floor division and modulo operation at the same time returning a two-tuple of the same type as the left hand side. For example:

In [21]:
s = pd.Series(np.arange(10))
print(s)

0    0
1    1
2    2
3    3
4    4
5    5
6    6
7    7
8    8
9    9
dtype: int32


In [22]:
div, rem = divmod(s, 3)
print(div)

0    0
1    0
2    0
3    1
4    1
5    1
6    2
7    2
8    2
9    3
dtype: int32


In [23]:
print(rem)

0    0
1    1
2    2
3    0
4    1
5    2
6    0
7    1
8    2
9    0
dtype: int32


In [24]:
idx = pd.Index(np.arange(10))
print(idx)

Int64Index([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], dtype='int64')


In [25]:
div, rem = divmod(idx, 3)
print(div)

Int64Index([0, 0, 0, 1, 1, 1, 2, 2, 2, 3], dtype='int64')


In [26]:
print(rem)

Int64Index([0, 1, 2, 0, 1, 2, 0, 1, 2, 0], dtype='int64')


We can also do elementwise divmod():

In [27]:
div, rem = divmod(s, [2, 2, 3, 3, 4, 4, 5, 5, 6, 6])
print(div)

0    0
1    0
2    0
3    1
4    1
5    1
6    1
7    1
8    1
9    1
dtype: int32


In [28]:
print(rem)

0    0
1    1
2    2
3    0
4    0
5    1
6    1
7    2
8    2
9    3
dtype: int32


### Missing data / operations with fill values
In Series and DataFrame, the arithmetic functions have the option of inputting a fill_value, namely a value to substitute when at most one of the values at a location are missing. For example, when adding two DataFrame objects, you may wish to treat NaN as 0 unless both DataFrames are missing that value, in which case the result will be NaN (you can later replace NaN with some other value using fillna if you wish).

In [29]:
print(df)

        one       two     three
a -0.005240  0.848099       NaN
b  1.323212 -0.196640  1.092488
c  0.380090 -0.416081  0.290022
d       NaN -0.788091 -1.715676


In [30]:
df2 = df.copy()
df2.iat[0, 2] = 1.0
print(df2)

        one       two     three
a -0.005240  0.848099  1.000000
b  1.323212 -0.196640  1.092488
c  0.380090 -0.416081  0.290022
d       NaN -0.788091 -1.715676


In [31]:
print(df + df2)

        one       two     three
a -0.010481  1.696198       NaN
b  2.646423 -0.393280  2.184977
c  0.760181 -0.832161  0.580044
d       NaN -1.576182 -3.431353


In [32]:
df.add(df2, fill_value=0)

,one,two,three
a,-0.010481,1.696198,1.000000
b,2.646423,-0.393280,2.184977
c,0.760181,-0.832161,0.580044
d,NaN,-1.576182,-3.431353


### Flexible comparisons
Series and DataFrame have the binary comparison methods eq, ne, lt, gt, le, and ge whose behavior is analogous to the binary arithmetic operations described above:

In [33]:
df.gt(df2)

,one,two,three
a,False,False,False
b,False,False,False
c,False,False,False
d,False,False,False


In [34]:
df2.ne(df)

,one,two,three
a,False,False,True
b,False,False,False
c,False,False,False
d,True,False,False


These operations produce a pandas object of the same type as the left-hand-side input that is of dtype bool. These boolean objects can be used in indexing operations, see the section on Boolean indexing.

### Boolean reductions
You can apply the reductions: empty, any(), all(), and bool() to provide a way to summarize a boolean result.

In [35]:
(df > 0).all()

one      False
two      False
three    False
dtype: bool

In [36]:
(df > 0).any()

one      True
two      True
three    True
dtype: bool

You can reduce to a final boolean value.

In [37]:
(df > 0).any().any()

True

You can test if a pandas object is empty, via the empty property.

In [38]:
df.empty

False

In [39]:
pd.DataFrame(columns=list("ABC")).empty

True

To evaluate single-element pandas objects in a boolean context, use the method bool():

In [40]:
pd.Series([True]).bool()

True

In [41]:
pd.Series([False]).bool()

False

In [42]:
pd.DataFrame([[True]]).bool()

True

In [43]:
pd.DataFrame([[False]]).bool()

False

>Warning
>
>You might be tempted to do the following:
```python
>>> if df:
...     pass
```
Or
```python
>>> df and df2
```
These will both raise errors, as you are trying to compare multiple values.:

```python
ValueError: The truth value of an array is ambiguous. Use a.empty, a.any() or a.all().
```
See gotchas for a more detailed discussion.

### Comparing if objects are equivalent
Often you may find that there is more than one way to compute the same result. As a simple example, consider df + df and df * 2. To test that these two computations produce the same result, given the tools shown above, you might imagine using (df + df == df * 2).all(). But in fact, this expression is False:

In [44]:
df + df == df * 2

,one,two,three
a,True,True,False
b,True,True,True
c,True,True,True
d,False,True,True


In [45]:
(df + df == df * 2).all()

one      False
two       True
three    False
dtype: bool

Notice that the boolean DataFrame df + df == df * 2 contains some False values! This is because NaNs do not compare as equals:

In [46]:
np.nan == np.nan

False

So, NDFrames (such as Series and DataFrames) have an equals() method for testing equality, with NaNs in corresponding locations treated as equal.

In [47]:
(df + df).equals(df * 2)

True

Note that the Series or DataFrame index needs to be in the same order for equality to be True:

In [48]:
df1 = pd.DataFrame({"col": ["foo", 0, np.nan]})

In [49]:
df2 = pd.DataFrame({"col": [np.nan, 0, "foo"]}, index=[2, 1, 0])

In [50]:
df1.equals(df2)

False

In [51]:
df1.equals(df2.sort_index())

True

### Comparing array-like objects
You can conveniently perform element-wise comparisons when comparing a pandas data structure with a scalar value:

In [52]:
pd.Series(["foo", "bar", "baz"]) == "foo"

0     True
1    False
2    False
dtype: bool

In [53]:
pd.Index(["foo", "bar", "baz"]) == "foo"

array([ True, False, False])

pandas also handles element-wise comparisons between different array-like objects of the same length:

In [54]:
pd.Series(["foo", "bar", "baz"]) == pd.Index(["foo", "bar", "qux"])

0     True
1     True
2    False
dtype: bool

In [55]:
pd.Series(["foo", "bar", "baz"]) == np.array(["foo", "bar", "qux"])

0     True
1     True
2    False
dtype: bool

Trying to compare Index or Series objects of different lengths will raise a ValueError:

In [56]:
import sys
try:
    pd.Series(['foo', 'bar', 'baz']) == pd.Series(['foo', 'bar'])
except:
    print("Unexpected error:", sys.exc_info()[0])

Unexpected error: <class 'ValueError'>


In [57]:
import sys
try:
    pd.Series(['foo', 'bar', 'baz']) == pd.Series(['foo'])
except:
    print("Unexpected error:", sys.exc_info()[0])

Unexpected error: <class 'ValueError'>


Note that this is different from the NumPy behavior where a comparison can be broadcast:

In [58]:
np.array([1, 2, 3]) == np.array([2])

array([False,  True, False])

or it can return False if broadcasting can not be done:

In [59]:
np.array([1, 2, 3]) == np.array([1, 2])

<ipython-input-59-7a76bc07877c>:1: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  np.array([1, 2, 3]) == np.array([1, 2])


False

### Combining overlapping data sets
A problem occasionally arising is the combination of two similar data sets where values in one are preferred over the other. An example would be two data series representing a particular economic indicator where one is considered to be of “higher quality”. However, the lower quality series might extend further back in history or have more complete data coverage. As such, we would like to combine two DataFrame objects where missing values in one DataFrame are conditionally filled with like-labeled values from the other DataFrame. The function implementing this operation is combine_first(), which we illustrate:

In [60]:
df1 = pd.DataFrame(
    {"A": [1.0, np.nan, 3.0, 5.0, np.nan], "B": [np.nan, 2.0, 3.0, np.nan, 6.0]}
)
print(df1)

     A    B
0  1.0  NaN
1  NaN  2.0
2  3.0  3.0
3  5.0  NaN
4  NaN  6.0


In [61]:
df2 = pd.DataFrame(
    {
        "A": [5.0, 2.0, 4.0, np.nan, 3.0, 7.0],
        "B": [np.nan, np.nan, 3.0, 4.0, 6.0, 8.0],        
    }
)
print(df2)

     A    B
0  5.0  NaN
1  2.0  NaN
2  4.0  3.0
3  NaN  4.0
4  3.0  6.0
5  7.0  8.0


In [62]:
df1.combine_first(df2)

,A,B
0,1.0,NaN
1,2.0,2.0
2,3.0,3.0
3,5.0,4.0
4,3.0,6.0
5,7.0,8.0


### General DataFrame combine
The combine_first() method above calls the more general DataFrame.combine(). This method takes another DataFrame and a combiner function, aligns the input DataFrame and then passes the combiner function pairs of Series (i.e., columns whose names are the same).

So, for instance, to reproduce combine_first() as above:

In [63]:
def combiner(x, y):
    return np.where(pd.isna(x), y, x)
df1.combine(df2, combiner)

,A,B
0,1.0,NaN
1,2.0,2.0
2,3.0,3.0
3,5.0,4.0
4,3.0,6.0
5,7.0,8.0


## Descriptive statistics
There exists a large number of methods for computing descriptive statistics and other related operations on Series, DataFrame. Most of these are aggregations (hence producing a lower-dimensional result) like sum(), mean(), and quantile(), but some of them, like cumsum() and cumprod(), produce an object of the same size. Generally speaking, these methods take an axis argument, just like ndarray.{sum, std, …}, but the axis can be specified by name or integer:

* **Series**: no axis argument needed
* **DataFrame**: “index” (axis=0, default), “columns” (axis=1)

For example:

In [64]:
df

,one,two,three
a,-0.005240,0.848099,NaN
b,1.323212,-0.196640,1.092488
c,0.380090,-0.416081,0.290022
d,NaN,-0.788091,-1.715676


In [65]:
df.mean(0)

one      0.566021
two     -0.138178
three   -0.111055
dtype: float64

In [66]:
df.mean(1)

a    0.421429
b    0.739687
c    0.084677
d   -1.251884
dtype: float64

All such methods have a skipna option signaling whether to exclude missing data (True by default):

In [67]:
df.sum(0, skipna=False)

one           NaN
two     -0.552712
three         NaN
dtype: float64

In [68]:
df.sum(axis=1, skipna=True)

a    0.842859
b    2.219060
c    0.254032
d   -2.503767
dtype: float64

Combined with the broadcasting / arithmetic behavior, one can describe various statistical procedures, like standardization (rendering data zero mean and standard deviation of 1), very concisely:

In [69]:
ts_stand = (df - df.mean()) / df.std()
ts_stand.std()

one      1.0
two      1.0
three    1.0
dtype: float64

In [70]:
xs_stand = df.sub(df.mean(1), axis=0).div(df.std(1), axis=0)
xs_stand.std(1)

a    1.0
b    1.0
c    1.0
d    1.0
dtype: float64

Note that methods like cumsum() and cumprod() preserve the location of NaN values. This is somewhat different from expanding() and rolling() since NaN behavior is furthermore dictated by a min_periods parameter.

In [71]:
df.cumsum()

,one,two,three
a,-0.005240,0.848099,NaN
b,1.317971,0.651459,1.092488
c,1.698062,0.235378,1.382510
d,NaN,-0.552712,-0.333166


Here is a quick reference summary table of common functions. Each also takes an optional level parameter which applies only if the object has a hierarchical index.

| Function | Description                                |
|----------|--------------------------------------------|
| count    | Number of non-NA observations              |
| sum      | Sum of values                              |
| mean     | Mean of values                             |
| mad      | Mean absolute deviation                    |
| median   | Arithmetic median of values                |
| min      | Minimum                                    |
| max      | Maximum                                    |
| mode     | Mode                                       |
| abs      | Absolute Value                             |
| prod     | Product of values                          |
| std      | Bessel-corrected sample standard deviation |
| var      | Unbiased variance                          |
| sem      | Standard error of the mean                 |
| skew     | Sample skewness (3rd moment)               |
| kurt     | Sample kurtosis (4th moment)               |
| quantile | Sample quantile (value at %)               |
| cumsum   | Cumulative sum                             |
| cumprod  | Cumulative product                         |
| cummax   | Cumulative maximum                         |
| cummin   | Cumulative minimum                         |

Note that by chance some NumPy methods, like mean, std, and sum, will exclude NAs on Series input by default:

In [72]:
np.mean(df["one"])

0.5660205155795461

In [73]:
np.mean(df["one"].to_numpy())

nan

Series.nunique() will return the number of unique non-NA values in a Series:

In [74]:
series = pd.Series(np.random.randn(500))
series[20:500] = np.nan
series[10:20] = 5
series.nunique()

11

### Summarizing data: describe
There is a convenient describe() function which computes a variety of summary statistics about a Series or the columns of a DataFrame (excluding NAs of course):

In [75]:
series = pd.Series(np.random.randn(1000))
series[::2] = np.nan
series.describe()

count    500.000000
mean      -0.079149
std        0.988353
min       -2.998327
25%       -0.744497
50%       -0.067590
75%        0.601400
max        4.153018
dtype: float64

In [76]:
frame = pd.DataFrame(np.random.randn(1000, 5), columns=["a", "b", "c", "d", "e"])
frame.iloc[::2] = np.nan
frame.describe()

,a,b,c,d,e
count,500.000000,500.000000,500.000000,500.000000,500.000000
mean,0.013807,-0.028207,-0.053324,0.058770,0.014350
std,1.024382,0.984543,0.964632,0.950461,1.025136
min,-2.669458,-3.348886,-2.870442,-2.896470,-2.840016
25%,-0.652367,-0.688053,-0.692198,-0.631094,-0.721327
50%,0.016301,-0.097869,-0.096552,-0.027811,-0.034451
75%,0.704286,0.640408,0.604031,0.677736,0.735065
max,2.797392,3.638798,2.593035,2.949649,3.344527


You can select specific percentiles to include in the output:

In [77]:
series.describe(percentiles=[0.05, 0.25, 0.75, 0.95])

count    500.000000
mean      -0.079149
std        0.988353
min       -2.998327
5%        -1.675816
25%       -0.744497
50%       -0.067590
75%        0.601400
95%        1.489278
max        4.153018
dtype: float64

By default, the median is always included.

For a non-numerical Series object, describe() will give a simple summary of the number of unique values and most frequently occurring values:

In [78]:
s = pd.Series(["a", "a", "b", "b", "a", "a", np.nan, "c", "d", "a"])
s.describe()

count     9
unique    4
top       a
freq      5
dtype: object

Note that on a mixed-type DataFrame object, describe() will restrict the summary to include only numerical columns or, if none are, only categorical columns:

In [79]:
frame = pd.DataFrame({"a": ["Yes", "Yes", "No", "No"], "b": range(4)})
frame.describe()

,b
count,4.000000
mean,1.500000
std,1.290994
min,0.000000
25%,0.750000
50%,1.500000
75%,2.250000
max,3.000000


This behavior can be controlled by providing a list of types as include/exclude arguments. The special value all can also be used:

In [80]:
frame.describe(include=["object"])

,a
count,4
unique,2
top,No
freq,2


In [81]:
frame.describe(include=["number"])

,b
count,4.000000
mean,1.500000
std,1.290994
min,0.000000
25%,0.750000
50%,1.500000
75%,2.250000
max,3.000000


In [82]:
frame.describe(include="all")

,a,b
count,4,4.000000
unique,2,NaN
top,No,NaN
freq,2,NaN
mean,NaN,1.500000
std,NaN,1.290994
min,NaN,0.000000
25%,NaN,0.750000
50%,NaN,1.500000
75%,NaN,2.250000


That feature relies on select_dtypes. Refer to there for details about accepted inputs.

### Index of min/max values
The idxmin() and idxmax() functions on Series and DataFrame compute the index labels with the minimum and maximum corresponding values:


In [83]:
s1 = pd.Series(np.random.randn(5))
s1

0   -0.374162
1    0.597173
2   -1.233083
3    0.790469
4    0.338724
dtype: float64

In [84]:
s1.idxmin(), s1.idxmax()

(2, 3)

In [85]:
df1 = pd.DataFrame(np.random.randn(5, 3), columns=["A", "B", "C"])
df1

,A,B,C
0,1.387497,1.697030,0.976915
1,-2.746141,2.255610,-0.233544
2,-1.473940,1.125703,-0.062572
3,1.162739,-3.027656,-0.423247
4,-1.944809,-1.246614,1.918486


In [86]:
df1.idxmin(axis=0)

A    1
B    3
C    3
dtype: int64

In [87]:
df1.idxmax(axis=1)

0    B
1    B
2    B
3    A
4    C
dtype: object

When there are multiple rows (or columns) matching the minimum or maximum value, idxmin() and idxmax() return the first matching index:

In [88]:
df3 = pd.DataFrame([2, 1, 1, 3, np.nan], columns=["A"], index=list("edcba"))
df3

,A
e,2.0
d,1.0
c,1.0
b,3.0
a,NaN


In [89]:
df3["A"].idxmin()

'd'

>Note
>idxmin and idxmax are called argmin and argmax in NumPy.

### Value counts (histogramming) / mode
The value_counts() Series method and top-level function computes a histogram of a 1D array of values. It can also be used as a function on regular arrays:

In [90]:
data = np.random.randint(0, 7, size=50)
data

array([6, 3, 1, 0, 1, 5, 6, 1, 6, 2, 6, 3, 0, 1, 3, 4, 2, 1, 6, 5, 4, 4,
       1, 6, 5, 0, 4, 3, 5, 5, 6, 0, 6, 6, 2, 6, 5, 0, 5, 5, 4, 4, 4, 0,
       0, 1, 0, 3, 3, 6])

In [91]:
s = pd.Series(data)
s.value_counts()

6    11
0     8
5     8
1     7
4     7
3     6
2     3
dtype: int64

In [92]:
pd.value_counts(data)

6    11
0     8
5     8
1     7
4     7
3     6
2     3
dtype: int64

*New in version 1.1.0.*

The value_counts() method can be used to count combinations across multiple columns. By default all columns are used but a subset can be selected using the subset argument.

In [93]:
data = {"a": [1, 2, 3, 4], "b": ["x", "x", "y", "y"]}
frame = pd.DataFrame(data)
frame.value_counts()

a  b
1  x    1
2  x    1
3  y    1
4  y    1
dtype: int64

Similarly, you can get the most frequently occurring value(s), i.e. the mode, of the values in a Series or DataFrame:

In [94]:
s5 = pd.Series([1, 1, 3, 3, 3, 5, 5, 7, 7, 7])
s5.mode()

0    3
1    7
dtype: int64

In [95]:
df5 = pd.DataFrame(
    {
        "A": np.random.randint(0, 7, size=50),
        "B": np.random.randint(-10, 15, size=50),        
    }
)
df5.mode()

,A,B
0,0,-3.0
1,2,NaN
2,3,NaN


### Discretization and quantiling
Continuous values can be discretized using the cut() (bins based on values) and qcut() (bins based on sample quantiles) functions:

In [96]:
arr = np.random.randn(20)
factor = pd.cut(arr, 4)
factor

[(-1.823, -1.049], (-1.049, -0.277], (-1.823, -1.049], (-0.277, 0.494], (-1.823, -1.049], ..., (-1.049, -0.277], (0.494, 1.266], (0.494, 1.266], (-1.823, -1.049], (0.494, 1.266]]
Length: 20
Categories (4, interval[float64]): [(-1.823, -1.049] < (-1.049, -0.277] < (-0.277, 0.494] < (0.494, 1.266]]

In [97]:
factor = pd.cut(arr, [-5, -1, 0, 1, 5])
factor

[(-5, -1], (-1, 0], (-5, -1], (0, 1], (-5, -1], ..., (-1, 0], (0, 1], (1, 5], (-5, -1], (1, 5]]
Length: 20
Categories (4, interval[int64]): [(-5, -1] < (-1, 0] < (0, 1] < (1, 5]]

qcut() computes sample quantiles. For example, we could slice up some normally distributed data into equal-size quartiles like so:

In [98]:
arr = np.random.randn(30)
factor = pd.qcut(arr, [0, 0.25, 0.5, 0.75, 1])
factor

[(0.224, 0.623], (-0.435, 0.224], (-1.8889999999999998, -0.435], (0.623, 3.056], (0.224, 0.623], ..., (-0.435, 0.224], (-1.8889999999999998, -0.435], (0.224, 0.623], (0.623, 3.056], (-0.435, 0.224]]
Length: 30
Categories (4, interval[float64]): [(-1.8889999999999998, -0.435] < (-0.435, 0.224] < (0.224, 0.623] < (0.623, 3.056]]

In [99]:
pd.value_counts(factor)

(-1.8889999999999998, -0.435]    8
(0.623, 3.056]                   8
(-0.435, 0.224]                  7
(0.224, 0.623]                   7
dtype: int64

We can also pass infinite values to define the bins:

In [100]:
arr = np.random.randn(20)
factor = pd.cut(arr, [-np.inf, 0, np.inf])
factor

[(0.0, inf], (0.0, inf], (0.0, inf], (0.0, inf], (0.0, inf], ..., (0.0, inf], (0.0, inf], (-inf, 0.0], (0.0, inf], (-inf, 0.0]]
Length: 20
Categories (2, interval[float64]): [(-inf, 0.0] < (0.0, inf]]

## Function application
To apply your own or another library’s functions to pandas objects, you should be aware of the three methods below. The appropriate method to use depends on whether your function expects to operate on an entire DataFrame or Series, row- or column-wise, or elementwise.

1. Tablewise Function Application: pipe()
2. Row or Column-wise Function Application: apply()
3. Aggregation API: agg() and transform()
4. Applying Elementwise Functions: applymap()

### Tablewise function application
DataFrames and Series can be passed into functions. However, if the function needs to be called in a chain, consider using the pipe() method.

First some setup:

In [101]:
def extract_city_name(df):
    """
    Chicago, IL -> Chicago for city_name column
    """
    df["city_name"] = df["city_and_code"].str.split(",").str.get(0)
    return df

def add_country_name(df, country_name=None):
    """
    Chicago -> Chicago-US for city_name column
    """
    col = "city_name"
    df["city_and_country"] = df[col] + country_name
    return df

df_p = pd.DataFrame({"city_and_code": ["Chicago, IL"]})

extract_city_name and add_country_name are functions taking and returning DataFrames.

Now compare the following:

In [102]:
add_country_name(extract_city_name(df_p), country_name="US")

,city_and_code,city_name,city_and_country
0,"Chicago, IL",Chicago,ChicagoUS


Is equivalent to:

In [103]:
df_p.pipe(extract_city_name).pipe(add_country_name, country_name="US")

,city_and_code,city_name,city_and_country
0,"Chicago, IL",Chicago,ChicagoUS


pandas encourages the second style, which is known as method chaining. pipe makes it easy to use your own or another library’s functions in method chains, alongside pandas’ methods.

In the example above, the functions extract_city_name and add_country_name each expected a DataFrame as the first positional argument. What if the function you wish to apply takes its data as, say, the second argument? In this case, provide pipe with a tuple of (callable, data_keyword). .pipe will route the DataFrame to the argument specified in the tuple.

For example, we can fit a regression using statsmodels. Their API expects a formula first and a DataFrame as the second argument, data. We pass in the function, keyword pair (sm.ols, 'data') to pipe:

In [104]:
import statsmodels.formula.api as sm
bb = pd.read_csv("data/baseball.csv", index_col="id")
(
    bb.query("h > 0")
    .assign(ln_h=lambda df: np.log(df.h))
    .pipe((sm.ols, "data"), "hr ~ ln_h + year + g + C(lg)")
    .fit()
    .summary()
)

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                     hr   R-squared:                       0.685
Model:                            OLS   Adj. R-squared:                  0.665
Method:                 Least Squares   F-statistic:                     34.28
Date:                Thu, 03 Jun 2021   Prob (F-statistic):           3.48e-15
Time:                        22:07:52   Log-Likelihood:                -205.92
No. Observations:                  68   AIC:                             421.8
Df Residuals:                      63   BIC:                             432.9
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
===============================================================================
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
Intercept   -8484.7720   4664.146     -1.819      0.074   -1.78e+04     835.780
C(lg)[T.NL]    -2.2736      1.325     -1.716      0.091      -4.922       0.375
ln_h           -1.3542      0.875     -1.547      0.127      -3.103       0.395
year            4.2277      2.324      1.819      0.074      -0.417       8.872
g               0.1841      0.029      6.258      0.000       0.125       0.243
==============================================================================
Omnibus:                       10.875   Durbin-Watson:                   1.999
Prob(Omnibus):                  0.004   Jarque-Bera (JB):               17.298
Skew:                           0.537   Prob(JB):                     0.000175
Kurtosis:                       5.225   Cond. No.                     1.49e+07
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.49e+07. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

The pipe method is inspired by unix pipes and more recently dplyr and magrittr, which have introduced the popular (%>%) (read pipe) operator for R. The implementation of pipe here is quite clean and feels right at home in Python. We encourage you to view the source code of pipe().

### Row or column-wise function application
Arbitrary functions can be applied along the axes of a DataFrame using the apply() method, which, like the descriptive statistics methods, takes an optional axis argument:

In [105]:
df.apply(np.mean)

one      0.566021
two     -0.138178
three   -0.111055
dtype: float64

In [106]:
df.apply(np.mean, axis=1)

a    0.421429
b    0.739687
c    0.084677
d   -1.251884
dtype: float64

In [107]:
df.apply(lambda x: x.max() - x.min())

one      1.328452
two      1.636190
three    2.808165
dtype: float64

In [108]:
df.apply(np.cumsum)

,one,two,three
a,-0.005240,0.848099,NaN
b,1.317971,0.651459,1.092488
c,1.698062,0.235378,1.382510
d,NaN,-0.552712,-0.333166


In [109]:
df.apply(np.exp)

,one,two,three
a,0.994773,2.335203,NaN
b,3.755463,0.821486,2.981684
c,1.462417,0.659627,1.336457
d,NaN,0.454712,0.179842


The apply() method will also dispatch on a string method name.

In [110]:
df.apply("mean")

one      0.566021
two     -0.138178
three   -0.111055
dtype: float64

In [111]:
df.apply("mean", axis=1)

a    0.421429
b    0.739687
c    0.084677
d   -1.251884
dtype: float64

The return type of the function passed to apply() affects the type of the final output from DataFrame.apply for the default behaviour:

* If the applied function returns a Series, the final output is a DataFrame. The columns match the index of the Series returned by the applied function.
* If the applied function returns any other type, the final output is a Series.

This default behaviour can be overridden using the result_type, which accepts three options: reduce, broadcast, and expand. These will determine how list-likes return values expand (or not) to a DataFrame.

apply() combined with some cleverness can be used to answer many questions about a data set. For example, suppose we wanted to extract the date where the maximum value for each column occurred:

In [112]:
tsdf = pd.DataFrame(
    np.random.randn(1000, 3),
    columns=["A", "B", "C"],
    index=pd.date_range("1/1/2000", periods=1000),
    )
tsdf.apply(lambda x: x.idxmax())

A   2001-08-03
B   2000-01-30
C   2001-05-21
dtype: datetime64[ns]

You may also pass additional arguments and keyword arguments to the apply() method. For instance, consider the following function you would like to apply:

In [113]:
def subtract_and_divide(x, sub, divide=1):
    return (x - sub) / divide

You may then apply this function as follows:

In [114]:
df.apply(subtract_and_divide, args=(5,), divide=3)

,one,two,three
a,-1.668413,-1.383967,NaN
b,-1.225596,-1.732213,-1.302504
c,-1.539970,-1.805360,-1.569993
d,NaN,-1.929364,-2.238559


Another useful feature is the ability to pass Series methods to carry out some Series operation on each column or row:

In [115]:
tsdf

,A,B,C
2000-01-01,-0.335902,1.493079,0.219948
2000-01-02,-0.263730,-0.495047,1.537886
2000-01-03,0.351397,-1.112626,0.015964
2000-01-04,-0.811811,0.195992,0.883682
2000-01-05,1.301142,-0.787303,1.615960
...,...,...,...
2002-09-22,-0.322511,-1.556428,0.874678
2002-09-23,0.603011,0.305450,-1.334022
2002-09-24,1.509251,-1.169521,0.186008
2002-09-25,-2.678630,1.231014,-0.845095


In [116]:
tsdf.apply(pd.Series.interpolate)

,A,B,C
2000-01-01,-0.335902,1.493079,0.219948
2000-01-02,-0.263730,-0.495047,1.537886
2000-01-03,0.351397,-1.112626,0.015964
2000-01-04,-0.811811,0.195992,0.883682
2000-01-05,1.301142,-0.787303,1.615960
...,...,...,...
2002-09-22,-0.322511,-1.556428,0.874678
2002-09-23,0.603011,0.305450,-1.334022
2002-09-24,1.509251,-1.169521,0.186008
2002-09-25,-2.678630,1.231014,-0.845095


Finally, apply() takes an argument raw which is False by default, which converts each row or column into a Series before applying the function. When set to True, the passed function will instead receive an ndarray object, which has positive performance implications if you do not need the indexing functionality.

### Aggregation API
The aggregation API allows one to express possibly multiple aggregation operations in a single concise way. This API is similar across pandas objects, see groupby API, the window API, and the resample API. The entry point for aggregation is DataFrame.aggregate(), or the alias DataFrame.agg().

We will use a similar starting frame from above:

In [117]:
tsdf = pd.DataFrame(
    np.random.randn(10, 3),
    columns=["A", "B", "C"],
    index=pd.date_range("1/1/2000", periods=10),
    )
tsdf.iloc[3:7] = np.nan
tsdf

,A,B,C
2000-01-01,0.483725,1.348574,-0.019953
2000-01-02,-2.095621,1.108451,-1.052461
2000-01-03,0.186139,-1.036476,-1.119160
2000-01-04,NaN,NaN,NaN
2000-01-05,NaN,NaN,NaN
2000-01-06,NaN,NaN,NaN
2000-01-07,NaN,NaN,NaN
2000-01-08,1.390345,-0.337125,-0.846664
2000-01-09,0.288159,-0.869943,-0.044898
2000-01-10,-0.968781,0.046088,-1.595831


Using a single function is equivalent to apply(). You can also pass named methods as strings. These will return a Series of the aggregated output:

In [118]:
tsdf.agg(np.sum)

A   -0.716034
B    0.259569
C   -4.678966
dtype: float64

In [119]:
tsdf.agg("sum")

A   -0.716034
B    0.259569
C   -4.678966
dtype: float64

In [120]:
# these are equivalent to a ``.sum()`` because we are aggregating
# on a single function
In [166]: tsdf.sum()

Single aggregations on a Series this will return a scalar value:

In [121]:
tsdf["A"].agg("sum")

-0.7160339652128991

### Aggregating with multiple functions
You can pass multiple aggregation arguments as a list. The results of each of the passed functions will be a row in the resulting DataFrame. These are naturally named from the aggregation function.

In [122]:
tsdf.agg(["sum"])

,A,B,C
sum,-0.716034,0.259569,-4.678966


Multiple functions yield multiple rows:

In [123]:
tsdf.agg(["sum", "mean"])

,A,B,C
sum,-0.716034,0.259569,-4.678966
mean,-0.119339,0.043262,-0.779828


On a Series, multiple functions return a Series, indexed by the function names:

In [124]:
tsdf["A"].agg(["sum", "mean"])

sum    -0.716034
mean   -0.119339
Name: A, dtype: float64

Passing a lambda function will yield a <lambda> named row:

In [125]:
tsdf["A"].agg(["sum", lambda x: x.mean()])

sum        -0.716034
<lambda>   -0.119339
Name: A, dtype: float64

Passing a named function will yield that name for the row:

In [126]:
def mymean(x):
    return x.mean()

tsdf["A"].agg(["sum", mymean])

sum      -0.716034
mymean   -0.119339
Name: A, dtype: float64

### Aggregating with a dict
Passing a dictionary of column names to a scalar or a list of scalars, to DataFrame.agg allows you to customize which functions are applied to which columns. Note that the results are not in any particular order, you can use an OrderedDict instead to guarantee ordering.

In [127]:
tsdf.agg({"A": "mean", "B": "sum"})

A   -0.119339
B    0.259569
dtype: float64

Passing a list-like will generate a DataFrame output. You will get a matrix-like output of all of the aggregators. The output will consist of all unique functions. Those that are not noted for a particular column will be NaN:

In [128]:
tsdf.agg({"A": ["mean", "min"], "B": "sum"})

,A,B
mean,-0.119339,NaN
min,-2.095621,NaN
sum,NaN,0.259569


### Mixed dtypes
When presented with mixed dtypes that cannot aggregate, .agg will only take the valid aggregations. This is similar to how .groupby.agg works.

In [129]:
mdf = pd.DataFrame(
    {
        "A": [1, 2, 3],
        "B": [1.0, 2.0, 3.0],
        "C": ["foo", "bar", "baz"],
        "D": pd.date_range("20130101", periods=3),
    }
)
mdf.dtypes

A             int64
B           float64
C            object
D    datetime64[ns]
dtype: object

In [130]:
mdf.agg(["min", "sum"])

,A,B,C,D
min,1,1.0,bar,2013-01-01
sum,6,6.0,foobarbaz,NaT


### Custom describe
With .agg() it is possible to easily create a custom describe function, similar to the built in describe function.

In [131]:
from functools import partial

q_25 = partial(pd.Series.quantile, q=0.25)
q_25.__name__ = "25%"
q_75 = partial(pd.Series.quantile, q=0.75)
q_75.__name__ = "75%"
tsdf.agg(["count", "mean", "std", "min", q_25, "median", q_75, "max"])

,A,B,C
count,6.000000,6.000000,6.000000
mean,-0.119339,0.043262,-0.779828
std,1.227147,0.998293,0.628862
min,-2.095621,-1.036476,-1.595831
25%,-0.680051,-0.736738,-1.102485
median,0.237149,-0.145518,-0.949562
75%,0.434834,0.842860,-0.245339
max,1.390345,1.348574,-0.019953


## Transform API
The transform() method returns an object that is indexed the same (same size) as the original. This API allows you to provide multiple operations at the same time rather than one-by-one. Its API is quite similar to the .agg API.

We create a frame similar to the one used in the above sections.

In [132]:
tsdf = pd.DataFrame(
    np.random.randn(10, 3),
    columns=["A", "B", "C"],
    index=pd.date_range("1/1/2000", periods=10),
)
tsdf.iloc[3:7] = np.nan
tsdf

,A,B,C
2000-01-01,1.274215,-1.170935,0.579068
2000-01-02,1.079663,0.791916,0.391517
2000-01-03,1.050935,-1.137028,1.011176
2000-01-04,NaN,NaN,NaN
2000-01-05,NaN,NaN,NaN
2000-01-06,NaN,NaN,NaN
2000-01-07,NaN,NaN,NaN
2000-01-08,1.228606,0.863057,-0.165071
2000-01-09,-0.106791,0.450969,-1.926701
2000-01-10,-0.349231,-1.080722,0.723320


Transform the entire frame. .transform() allows input functions as: a NumPy function, a string function name or a user defined function.

In [133]:
tsdf.transform(np.abs)

,A,B,C
2000-01-01,1.274215,1.170935,0.579068
2000-01-02,1.079663,0.791916,0.391517
2000-01-03,1.050935,1.137028,1.011176
2000-01-04,NaN,NaN,NaN
2000-01-05,NaN,NaN,NaN
2000-01-06,NaN,NaN,NaN
2000-01-07,NaN,NaN,NaN
2000-01-08,1.228606,0.863057,0.165071
2000-01-09,0.106791,0.450969,1.926701
2000-01-10,0.349231,1.080722,0.723320


In [134]:
tsdf.transform(np.abs)

,A,B,C
2000-01-01,1.274215,1.170935,0.579068
2000-01-02,1.079663,0.791916,0.391517
2000-01-03,1.050935,1.137028,1.011176
2000-01-04,NaN,NaN,NaN
2000-01-05,NaN,NaN,NaN
2000-01-06,NaN,NaN,NaN
2000-01-07,NaN,NaN,NaN
2000-01-08,1.228606,0.863057,0.165071
2000-01-09,0.106791,0.450969,1.926701
2000-01-10,0.349231,1.080722,0.723320


In [135]:
tsdf.transform("abs")

,A,B,C
2000-01-01,1.274215,1.170935,0.579068
2000-01-02,1.079663,0.791916,0.391517
2000-01-03,1.050935,1.137028,1.011176
2000-01-04,NaN,NaN,NaN
2000-01-05,NaN,NaN,NaN
2000-01-06,NaN,NaN,NaN
2000-01-07,NaN,NaN,NaN
2000-01-08,1.228606,0.863057,0.165071
2000-01-09,0.106791,0.450969,1.926701
2000-01-10,0.349231,1.080722,0.723320


In [136]:
tsdf.transform(lambda x: x.abs())

,A,B,C
2000-01-01,1.274215,1.170935,0.579068
2000-01-02,1.079663,0.791916,0.391517
2000-01-03,1.050935,1.137028,1.011176
2000-01-04,NaN,NaN,NaN
2000-01-05,NaN,NaN,NaN
2000-01-06,NaN,NaN,NaN
2000-01-07,NaN,NaN,NaN
2000-01-08,1.228606,0.863057,0.165071
2000-01-09,0.106791,0.450969,1.926701
2000-01-10,0.349231,1.080722,0.723320


Here transform() received a single function; this is equivalent to a ufunc application.

In [137]:
np.abs(tsdf)

,A,B,C
2000-01-01,1.274215,1.170935,0.579068
2000-01-02,1.079663,0.791916,0.391517
2000-01-03,1.050935,1.137028,1.011176
2000-01-04,NaN,NaN,NaN
2000-01-05,NaN,NaN,NaN
2000-01-06,NaN,NaN,NaN
2000-01-07,NaN,NaN,NaN
2000-01-08,1.228606,0.863057,0.165071
2000-01-09,0.106791,0.450969,1.926701
2000-01-10,0.349231,1.080722,0.723320


Passing a single function to .transform() with a Series will yield a single Series in return.

In [138]:
tsdf["A"].transform(np.abs)

2000-01-01    1.274215
2000-01-02    1.079663
2000-01-03    1.050935
2000-01-04         NaN
2000-01-05         NaN
2000-01-06         NaN
2000-01-07         NaN
2000-01-08    1.228606
2000-01-09    0.106791
2000-01-10    0.349231
Freq: D, Name: A, dtype: float64

### Transform with multiple functions
Passing multiple functions will yield a column MultiIndexed DataFrame. The first level will be the original frame column names; the second level will be the names of the transforming functions.

In [139]:
tsdf.transform([np.abs, lambda x: x + 1])

A                   B                   C          
            absolute  <lambda>  absolute  <lambda>  absolute  <lambda>
2000-01-01  1.274215  2.274215  1.170935 -0.170935  0.579068  1.579068
2000-01-02  1.079663  2.079663  0.791916  1.791916  0.391517  1.391517
2000-01-03  1.050935  2.050935  1.137028 -0.137028  1.011176  2.011176
2000-01-04       NaN       NaN       NaN       NaN       NaN       NaN
2000-01-05       NaN       NaN       NaN       NaN       NaN       NaN
2000-01-06       NaN       NaN       NaN       NaN       NaN       NaN
2000-01-07       NaN       NaN       NaN       NaN       NaN       NaN
2000-01-08  1.228606  2.228606  0.863057  1.863057  0.165071  0.834929
2000-01-09  0.106791  0.893209  0.450969  1.450969  1.926701 -0.926701
2000-01-10  0.349231  0.650769  1.080722 -0.080722  0.723320  1.723320

Passing multiple functions to a Series will yield a DataFrame. The resulting column names will be the transforming functions.

In [140]:
tsdf["A"].transform([np.abs, lambda x: x + 1])

,absolute,<lambda>
2000-01-01,1.274215,2.274215
2000-01-02,1.079663,2.079663
2000-01-03,1.050935,2.050935
2000-01-04,NaN,NaN
2000-01-05,NaN,NaN
2000-01-06,NaN,NaN
2000-01-07,NaN,NaN
2000-01-08,1.228606,2.228606
2000-01-09,0.106791,0.893209
2000-01-10,0.349231,0.650769


### Transforming with a dict
Passing a dict of functions will allow selective transforming per column.

In [141]:
tsdf.transform({"A": np.abs, "B": lambda x: x + 1})

,A,B
2000-01-01,1.274215,-0.170935
2000-01-02,1.079663,1.791916
2000-01-03,1.050935,-0.137028
2000-01-04,NaN,NaN
2000-01-05,NaN,NaN
2000-01-06,NaN,NaN
2000-01-07,NaN,NaN
2000-01-08,1.228606,1.863057
2000-01-09,0.106791,1.450969
2000-01-10,0.349231,-0.080722


Passing a dict of lists will generate a MultiIndexed DataFrame with these selective transforms.

In [142]:
tsdf.transform({"A": np.abs, "B": [lambda x: x + 1, "sqrt"]})

C:\bin\anaconda3\envs\eda_dev39\lib\site-packages\pandas\core\arraylike.py:358: RuntimeWarning: invalid value encountered in sqrt
  result = getattr(ufunc, method)(*inputs, **kwargs)


A         B          
            absolute  <lambda>      sqrt
2000-01-01  1.274215 -0.170935       NaN
2000-01-02  1.079663  1.791916  0.889897
2000-01-03  1.050935 -0.137028       NaN
2000-01-04       NaN       NaN       NaN
2000-01-05       NaN       NaN       NaN
2000-01-06       NaN       NaN       NaN
2000-01-07       NaN       NaN       NaN
2000-01-08  1.228606  1.863057  0.929009
2000-01-09  0.106791  1.450969  0.671542
2000-01-10  0.349231 -0.080722       NaN

## Applying elementwise functions
Since not all functions can be vectorized (accept NumPy arrays and return another array or value), the methods applymap() on DataFrame and analogously map() on Series accept any Python function taking a single value and returning a single value. For example:

In [143]:
df4 = pd.DataFrame(
    {
        "one": [1.394981,  0.343054, 0.695246, np.NaN],
        "two": [1.772517,  1.912123, 1.478369, 0.279344],
        "three": [np.NaN, -0.050390, 1.227435, -0.613172]
    },
    index = ["a", "b", "c", "c"]
)
print(df4)

        one       two     three
a  1.394981  1.772517       NaN
b  0.343054  1.912123 -0.050390
c  0.695246  1.478369  1.227435
c       NaN  0.279344 -0.613172


In [144]:
def f(x):
    return len(str(x))
df4["one"].map(f)

a    8
b    8
c    8
c    3
Name: one, dtype: int64

In [145]:
df4.applymap(f)

,one,two,three
a,8,8,3
b,8,8,8
c,8,8,8
c,3,8,9


Series.map() has an additional feature; it can be used to easily “link” or “map” values defined by a secondary series. This is closely related to merging/joining functionality:

In [146]:
s = pd.Series(
    ["six", "seven", "six", "seven", "six"], index=["a", "b", "c", "d", "e"]
)
t = pd.Series({"six": 6.0, "seven": 7.0})
s

a      six
b    seven
c      six
d    seven
e      six
dtype: object

In [147]:
s.map(t)

a    6.0
b    7.0
c    6.0
d    7.0
e    6.0
dtype: float64

## Reindexing and altering labels
reindex() is the fundamental data alignment method in pandas. It is used to implement nearly all other features relying on label-alignment functionality. To reindex means to conform the data to match a given set of labels along a particular axis. This accomplishes several things:

* Reorders the existing data to match a new set of labels
* Inserts missing value (NA) markers in label locations where no data for that label existed
* If specified, fill data for missing labels using logic (highly relevant to working with time series data)

Here is a simple example:

In [148]:
s = pd.Series(np.random.randn(5), index=["a", "b", "c", "d", "e"])
s

a   -0.243565
b    0.952286
c   -1.490773
d    1.074369
e   -0.235642
dtype: float64

In [149]:
s.reindex(["e", "b", "f", "d"])

e   -0.235642
b    0.952286
f         NaN
d    1.074369
dtype: float64

Here, the f label was not contained in the Series and hence appears as NaN in the result.

With a DataFrame, you can simultaneously reindex the index and columns:

In [150]:
df

,one,two,three
a,-0.005240,0.848099,NaN
b,1.323212,-0.196640,1.092488
c,0.380090,-0.416081,0.290022
d,NaN,-0.788091,-1.715676


In [151]:
df.reindex(index=["c", "f", "b"], columns=["three", "two", "one"])

,three,two,one
c,0.290022,-0.416081,0.380090
f,NaN,NaN,NaN
b,1.092488,-0.196640,1.323212


You may also use reindex with an axis keyword:

In [152]:
df.reindex(["c", "f", "b"], axis="index")

,one,two,three
c,0.380090,-0.416081,0.290022
f,NaN,NaN,NaN
b,1.323212,-0.196640,1.092488


Note that the Index objects containing the actual axis labels can be shared between objects. So if we have a Series and a DataFrame, the following can be done:

In [153]:
rs = s.reindex(df.index)
rs

a   -0.243565
b    0.952286
c   -1.490773
d    1.074369
dtype: float64

In [154]:
rs.index is df.index

True

This means that the reindexed Series’s index is the same Python object as the DataFrame’s index.

DataFrame.reindex() also supports an “axis-style” calling convention, where you specify a single labels argument and the axis it applies to.

In [155]:
df.reindex(["c", "f", "b"], axis="index")

,one,two,three
c,0.380090,-0.416081,0.290022
f,NaN,NaN,NaN
b,1.323212,-0.196640,1.092488


In [156]:
df.reindex(["three", "two", "one"], axis="columns")

,three,two,one
a,NaN,0.848099,-0.005240
b,1.092488,-0.196640,1.323212
c,0.290022,-0.416081,0.380090
d,-1.715676,-0.788091,NaN


> See also
> MultiIndex / Advanced Indexing is an even more concise way of doing reindexing.

> Note
> When writing performance-sensitive code, there is a good reason to spend some time becoming a reindexing ninja: many operations are faster on pre-aligned data. Adding two unaligned DataFrames internally triggers a reindexing step. For exploratory analysis you will hardly notice the difference (because reindex has been heavily optimized), but when CPU cycles matter sprinkling a few explicit reindex calls here and there can have an impact.

### Reindexing to align with another object
You may wish to take an object and reindex its axes to be labeled the same as another object. While the syntax for this is straightforward albeit verbose, it is a common enough operation that the reindex_like() method is available to make this simpler:

In [157]:
df2 = pd.DataFrame(
    np.random.randn(3, 2),
    columns=["one", "two"],
    index = ["a", "b", "c"]
)
print(df2)

        one       two
a -0.600836 -1.132610
b  1.036710 -1.777501
c -0.461130 -1.004266


In [158]:
df3 = pd.DataFrame(
    np.random.randn(3, 2),
    columns=["one", "two"],
    index = ["a", "b", "c"]
)
print(df3)

        one       two
a -0.723013  0.630871
b  1.455664 -0.616874
c -0.394000 -1.208772


In [159]:
df.reindex_like(df2)

,one,two
a,-0.005240,0.848099
b,1.323212,-0.196640
c,0.380090,-0.416081


### Aligning objects with each other with align
The align() method is the fastest way to simultaneously align two objects. It supports a join argument (related to joining and merging):

* join='outer': take the union of the indexes (default)
* join='left': use the calling object’s index
* join='right': use the passed object’s index
* join='inner': intersect the indexes

It returns a tuple with both of the reindexed Series:

In [160]:
s = pd.Series(np.random.randn(5), index=["a", "b", "c", "d", "e"])
s1 = s[:4]
s2 = s[1:]
s1.align(s2)

(a    1.279137
 b    0.655367
 c    0.401400
 d    0.872943
 e         NaN
 dtype: float64,
 a         NaN
 b    0.655367
 c    0.401400
 d    0.872943
 e   -0.507748
 dtype: float64)

In [161]:
s1.align(s2, join="inner")

(b    0.655367
 c    0.401400
 d    0.872943
 dtype: float64,
 b    0.655367
 c    0.401400
 d    0.872943
 dtype: float64)

In [162]:
s1.align(s2, join="left")

(a    1.279137
 b    0.655367
 c    0.401400
 d    0.872943
 dtype: float64,
 a         NaN
 b    0.655367
 c    0.401400
 d    0.872943
 dtype: float64)

For DataFrames, the join method will be applied to both the index and the columns by default:

In [163]:
df.align(df2, join="inner")

(        one       two
 a -0.005240  0.848099
 b  1.323212 -0.196640
 c  0.380090 -0.416081,
         one       two
 a -0.600836 -1.132610
 b  1.036710 -1.777501
 c -0.461130 -1.004266)

You can also pass an axis option to only align on the specified axis:

In [164]:
df.align(df2, join="inner", axis=0)

(        one       two     three
 a -0.005240  0.848099       NaN
 b  1.323212 -0.196640  1.092488
 c  0.380090 -0.416081  0.290022,
         one       two
 a -0.600836 -1.132610
 b  1.036710 -1.777501
 c -0.461130 -1.004266)

If you pass a Series to DataFrame.align(), you can choose to align both objects either on the DataFrame’s index or columns using the axis argument:

In [165]:
df.align(df2.iloc[0], axis=1)

(        one     three       two
 a -0.005240       NaN  0.848099
 b  1.323212  1.092488 -0.196640
 c  0.380090  0.290022 -0.416081
 d       NaN -1.715676 -0.788091,
 one     -0.600836
 three         NaN
 two     -1.132610
 Name: a, dtype: float64)

### Filling while reindexing
reindex() takes an optional parameter method which is a filling method chosen from the following table:

| Method           | Action                            |
|------------------|-----------------------------------|
|pad / ffill       | Fill values forward               |
| bfill / backfill | Fill values backward              |
| nearest          | Fill from the nearest index value |

We illustrate these fill methods on a simple Series:

In [166]:
rng = pd.date_range("1/3/2000", periods=8)
ts = pd.Series(np.random.randn(8), index=rng)
ts2 = ts[[0, 3, 6]]
ts

2000-01-03    0.373411
2000-01-04    0.889825
2000-01-05    1.649223
2000-01-06   -1.387402
2000-01-07    1.332058
2000-01-08   -0.120217
2000-01-09   -0.306842
2000-01-10   -0.167877
Freq: D, dtype: float64

In [167]:
ts2

2000-01-03    0.373411
2000-01-06   -1.387402
2000-01-09   -0.306842
Freq: 3D, dtype: float64

In [168]:
ts2.reindex(ts.index)

2000-01-03    0.373411
2000-01-04         NaN
2000-01-05         NaN
2000-01-06   -1.387402
2000-01-07         NaN
2000-01-08         NaN
2000-01-09   -0.306842
2000-01-10         NaN
Freq: D, dtype: float64

In [169]:
ts2.reindex(ts.index, method="ffill")

2000-01-03    0.373411
2000-01-04    0.373411
2000-01-05    0.373411
2000-01-06   -1.387402
2000-01-07   -1.387402
2000-01-08   -1.387402
2000-01-09   -0.306842
2000-01-10   -0.306842
Freq: D, dtype: float64

In [170]:
ts2.reindex(ts.index, method="bfill")

2000-01-03    0.373411
2000-01-04   -1.387402
2000-01-05   -1.387402
2000-01-06   -1.387402
2000-01-07   -0.306842
2000-01-08   -0.306842
2000-01-09   -0.306842
2000-01-10         NaN
Freq: D, dtype: float64

In [171]:
ts2.reindex(ts.index, method="nearest")

2000-01-03    0.373411
2000-01-04    0.373411
2000-01-05   -1.387402
2000-01-06   -1.387402
2000-01-07   -1.387402
2000-01-08   -0.306842
2000-01-09   -0.306842
2000-01-10   -0.306842
Freq: D, dtype: float64

These methods require that the indexes are ordered increasing or decreasing.

Note that the same result could have been achieved using fillna (except for method='nearest') or interpolate:

In [172]:
ts2.reindex(ts.index).fillna(method="ffill")

2000-01-03    0.373411
2000-01-04    0.373411
2000-01-05    0.373411
2000-01-06   -1.387402
2000-01-07   -1.387402
2000-01-08   -1.387402
2000-01-09   -0.306842
2000-01-10   -0.306842
Freq: D, dtype: float64

reindex() will raise a ValueError if the index is not monotonically increasing or decreasing. fillna() and interpolate() will not perform any checks on the order of the index.

### Limits on filling while reindexing
The limit and tolerance arguments provide additional control over filling while reindexing. Limit specifies the maximum count of consecutive matches:

In [173]:
ts2.reindex(ts.index, method="ffill", limit=1)

2000-01-03    0.373411
2000-01-04    0.373411
2000-01-05         NaN
2000-01-06   -1.387402
2000-01-07   -1.387402
2000-01-08         NaN
2000-01-09   -0.306842
2000-01-10   -0.306842
Freq: D, dtype: float64

In contrast, tolerance specifies the maximum distance between the index and indexer values:

In [174]:
ts2.reindex(ts.index, method="ffill", tolerance="1 day")

2000-01-03    0.373411
2000-01-04    0.373411
2000-01-05         NaN
2000-01-06   -1.387402
2000-01-07   -1.387402
2000-01-08         NaN
2000-01-09   -0.306842
2000-01-10   -0.306842
Freq: D, dtype: float64

Notice that when used on a DatetimeIndex, TimedeltaIndex or PeriodIndex, tolerance will coerced into a Timedelta if possible. This allows you to specify tolerance with appropriate strings.

### Dropping labels from an axis
A method closely related to reindex is the drop() function. It removes a set of labels from an axis:

In [175]:
df

,one,two,three
a,-0.005240,0.848099,NaN
b,1.323212,-0.196640,1.092488
c,0.380090,-0.416081,0.290022
d,NaN,-0.788091,-1.715676


In [176]:
df.drop(["a", "d"], axis=0)

,one,two,three
b,1.323212,-0.196640,1.092488
c,0.380090,-0.416081,0.290022


In [177]:
df.drop(["one"], axis=1)

,two,three
a,0.848099,NaN
b,-0.196640,1.092488
c,-0.416081,0.290022
d,-0.788091,-1.715676


Note that the following also works, but is a bit less obvious / clean:

In [178]:
df.reindex(df.index.difference(["a", "d"]))

,one,two,three
b,1.323212,-0.196640,1.092488
c,0.380090,-0.416081,0.290022


### Renaming / mapping labels
The rename() method allows you to relabel an axis based on some mapping (a dict or Series) or an arbitrary function.

In [179]:
s

a    1.279137
b    0.655367
c    0.401400
d    0.872943
e   -0.507748
dtype: float64

In [180]:
s.rename(str.upper)

A    1.279137
B    0.655367
C    0.401400
D    0.872943
E   -0.507748
dtype: float64

If you pass a function, it must return a value when called with any of the labels (and must produce a set of unique values). A dict or Series can also be used:

In [181]:
df.rename(
    columns={"one": "foo", "two": "bar"},
    index={"a": "apple", "b": "banana", "d": "durian"},
)

,foo,bar,three
apple,-0.005240,0.848099,NaN
banana,1.323212,-0.196640,1.092488
c,0.380090,-0.416081,0.290022
durian,NaN,-0.788091,-1.715676


If the mapping doesn’t include a column/index label, it isn’t renamed. Note that extra labels in the mapping don’t throw an error.

DataFrame.rename() also supports an “axis-style” calling convention, where you specify a single mapper and the axis to apply that mapping to.

In [182]:
df.rename({"one": "foo", "two": "bar"}, axis="columns")

,foo,bar,three
a,-0.005240,0.848099,NaN
b,1.323212,-0.196640,1.092488
c,0.380090,-0.416081,0.290022
d,NaN,-0.788091,-1.715676


In [183]:
df.rename({"a": "apple", "b": "banana", "d": "durian"}, axis="index")

,one,two,three
apple,-0.005240,0.848099,NaN
banana,1.323212,-0.196640,1.092488
c,0.380090,-0.416081,0.290022
durian,NaN,-0.788091,-1.715676


The rename() method also provides an inplace named parameter that is by default False and copies the underlying data. Pass inplace=True to rename the data in place.

Finally, rename() also accepts a scalar or list-like for altering the Series.name attribute.

In [184]:
s.rename("scalar-name")

a    1.279137
b    0.655367
c    0.401400
d    0.872943
e   -0.507748
Name: scalar-name, dtype: float64

*New in version 0.24.0.*

The methods DataFrame.rename_axis() and Series.rename_axis() allow specific names of a MultiIndex to be changed (as opposed to the labels).

In [185]:
df = pd.DataFrame(
    {"x": [1, 2, 3, 4, 5, 6], "y": [10, 20, 30, 40, 50, 60]},
    index=pd.MultiIndex.from_product(
        [["a", "b", "c"], [1, 2]], names=["let", "num"]
    ),
)
df

x   y
let num       
a   1    1  10
    2    2  20
b   1    3  30
    2    4  40
c   1    5  50
    2    6  60

In [186]:
df.rename_axis(index={"let": "abc"})

x   y
abc num       
a   1    1  10
    2    2  20
b   1    3  30
    2    4  40
c   1    5  50
    2    6  60

In [187]:
df.rename_axis(index=str.upper)

x   y
LET NUM       
a   1    1  10
    2    2  20
b   1    3  30
    2    4  40
c   1    5  50
    2    6  60

## Iteration
The behavior of basic iteration over pandas objects depends on the type. When iterating over a Series, it is regarded as array-like, and basic iteration produces the values. DataFrames follow the dict-like convention of iterating over the “keys” of the objects.

In short, basic iteration (for i in object) produces:

* Series: values
* DataFrame: column labels

Thus, for example, iterating over a DataFrame gives you the column names:

In [188]:
df = pd.DataFrame(
    {"col1": np.random.randn(3), "col2": np.random.randn(3)}, index=["a", "b", "c"]
)

for col in df:
    print(col)

col1
col2


pandas objects also have the dict-like items() method to iterate over the (key, value) pairs.

To iterate over the rows of a DataFrame, you can use the following methods:

* iterrows(): Iterate over the rows of a DataFrame as (index, Series) pairs. This converts the rows to Series objects, which can change the dtypes and has some performance implications.
* itertuples(): Iterate over the rows of a DataFrame as namedtuples of the values. This is a lot faster than iterrows(), and is in most cases preferable to use to iterate over the values of a DataFrame.

> **Warning**
> Iterating through pandas objects is generally slow. In many cases, iterating manually over the rows is not needed and can be avoided with one of the following approaches:
> 
> * Look for a vectorized solution: many operations can be performed using built-in methods or NumPy functions, (boolean) indexing, …
> * When you have a function that cannot work on the full DataFrame/Series at once, it is better to use apply() instead of iterating over the values. See the docs on function application.
> * If you need to do iterative manipulations on the values but performance is important, consider writing the inner loop with cython or numba. See the enhancing performance section for some examples of this approach.


> **Warning**
> You should never modify something you are iterating over. This is not guaranteed to work in all cases. Depending on the data types, the iterator returns a copy and not a view, and writing to it will have no effect!
> 
> For example, in the following case setting the value has no effect:


In [189]:
df = pd.DataFrame({"a": [1, 2, 3], "b": ["a", "b", "c"]})
print(df)

   a  b
0  1  a
1  2  b
2  3  c


In [190]:
for index, row in df.iterrows():
    row["a"] = 10
print(df)

   a  b
0  1  a
1  2  b
2  3  c


df is not changed!

### items
Consistent with the dict-like interface, items() iterates through key-value pairs:

* Series: (index, scalar value) pairs
* DataFrame: (column, Series) pairs

For example:

In [191]:
for label, ser in df.items():
    print(label)
    print(ser)

a
0    1
1    2
2    3
Name: a, dtype: int64
b
0    a
1    b
2    c
Name: b, dtype: object


### iterrows
iterrows() allows you to iterate through the rows of a DataFrame as Series objects. It returns an iterator yielding each index value along with a Series containing the data in each row:

In [192]:
for row_index, row in df.iterrows():
    print(row_index, row, sep="\n")

0
a    1
b    a
Name: 0, dtype: object
1
a    2
b    b
Name: 1, dtype: object
2
a    3
b    c
Name: 2, dtype: object


> Note
>
> Because iterrows() returns a Series for each row, it does not preserve dtypes across the rows (dtypes are preserved across columns for DataFrames). For example,

In [193]:
df_orig = pd.DataFrame([[1, 1.5]], columns=["int", "float"])
df_orig.dtypes

int        int64
float    float64
dtype: object

In [194]:
row = next(df_orig.iterrows())[1]
row

int      1.0
float    1.5
Name: 0, dtype: float64

All values in row, returned as a Series, are now upcasted to floats, also the original integer value in column x:

In [195]:
row["int"].dtype

dtype('float64')

In [196]:
df_orig["int"].dtype

dtype('int64')

To preserve dtypes while iterating over the rows, it is better to use itertuples() which returns namedtuples of the values and which is generally much faster than iterrows().

For instance, a contrived way to transpose the DataFrame would be:

In [197]:
df2 = pd.DataFrame({"x": [1, 2, 3], "y": [4, 5, 6]})
print(df2)

   x  y
0  1  4
1  2  5
2  3  6


In [198]:
print(df2.T)

   0  1  2
x  1  2  3
y  4  5  6


In [199]:
df2_t = pd.DataFrame({idx: values for idx, values in df2.iterrows()})
print(df2_t)

   0  1  2
x  1  2  3
y  4  5  6


### itertuples
The itertuples() method will return an iterator yielding a namedtuple for each row in the DataFrame. The first element of the tuple will be the row’s corresponding index value, while the remaining values are the row values.

For instance:

In [200]:
for row in df.itertuples():
    print(row)

Pandas(Index=0, a=1, b='a')
Pandas(Index=1, a=2, b='b')
Pandas(Index=2, a=3, b='c')


This method does not convert the row to a Series object; it merely returns the values inside a namedtuple. Therefore, itertuples() preserves the data type of the values and is generally faster as iterrows().

> Note
>
>The column names will be renamed to positional names if they are invalid Python identifiers, repeated, or start with an underscore. With a large number of columns (>255), regular tuples are returned.

## .dt accessor
Series has an accessor to succinctly return datetime like properties for the values of the Series, if it is a datetime/period like Series. This will return a Series, indexed like the existing Series.

In [201]:
# datetime
s = pd.Series(pd.date_range("20130101 09:10:12", periods=4))
s

0   2013-01-01 09:10:12
1   2013-01-02 09:10:12
2   2013-01-03 09:10:12
3   2013-01-04 09:10:12
dtype: datetime64[ns]

In [202]:
s.dt.hour

0    9
1    9
2    9
3    9
dtype: int64

In [203]:
s.dt.second

0    12
1    12
2    12
3    12
dtype: int64

In [204]:
s.dt.day

0    1
1    2
2    3
3    4
dtype: int64

This enables nice expressions like this:

In [205]:
s[s.dt.day == 2]

1   2013-01-02 09:10:12
dtype: datetime64[ns]

You can easily produces tz aware transformations:

In [206]:
stz = s.dt.tz_localize("US/Eastern")
stz

0   2013-01-01 09:10:12-05:00
1   2013-01-02 09:10:12-05:00
2   2013-01-03 09:10:12-05:00
3   2013-01-04 09:10:12-05:00
dtype: datetime64[ns, US/Eastern]

In [207]:
stz.dt.tz

<DstTzInfo 'US/Eastern' LMT-1 day, 19:04:00 STD>

You can also chain these types of operations:

In [208]:
s.dt.tz_localize("UTC").dt.tz_convert("US/Eastern")

0   2013-01-01 04:10:12-05:00
1   2013-01-02 04:10:12-05:00
2   2013-01-03 04:10:12-05:00
3   2013-01-04 04:10:12-05:00
dtype: datetime64[ns, US/Eastern]

You can also format datetime values as strings with Series.dt.strftime() which supports the same format as the standard strftime().

In [209]:
# DatetimeIndex
s = pd.Series(pd.date_range("20130101", periods=4))
s

0   2013-01-01
1   2013-01-02
2   2013-01-03
3   2013-01-04
dtype: datetime64[ns]

In [210]:
s.dt.tz_localize("UTC").dt.tz_convert("US/Eastern")

0   2012-12-31 19:00:00-05:00
1   2013-01-01 19:00:00-05:00
2   2013-01-02 19:00:00-05:00
3   2013-01-03 19:00:00-05:00
dtype: datetime64[ns, US/Eastern]

You can also format datetime values as strings with Series.dt.strftime() which supports the same format as the standard strftime().

In [211]:
# DatetimeIndex
s = pd.Series(pd.date_range("20130101", periods=4))
s

0   2013-01-01
1   2013-01-02
2   2013-01-03
3   2013-01-04
dtype: datetime64[ns]

In [212]:
s.dt.strftime("%Y/%m/%d")

0    2013/01/01
1    2013/01/02
2    2013/01/03
3    2013/01/04
dtype: object

In [213]:
# PeriodIndex
s = pd.Series(pd.period_range("20130101", periods=4))
s

0    2013-01-01
1    2013-01-02
2    2013-01-03
3    2013-01-04
dtype: period[D]

In [214]:
s.dt.strftime("%Y/%m/%d")

0    2013/01/01
1    2013/01/02
2    2013/01/03
3    2013/01/04
dtype: object

The .dt accessor works for period and timedelta dtypes.

In [215]:
# period
s = pd.Series(pd.period_range("20130101", periods=4, freq="D"))
s

0    2013-01-01
1    2013-01-02
2    2013-01-03
3    2013-01-04
dtype: period[D]

In [216]:
s.dt.year

0    2013
1    2013
2    2013
3    2013
dtype: int64

In [217]:
s.dt.day

0    1
1    2
2    3
3    4
dtype: int64

In [218]:
# timedelta
s = pd.Series(pd.timedelta_range("1 day 00:00:05", periods=4, freq="s"))
s

0   1 days 00:00:05
1   1 days 00:00:06
2   1 days 00:00:07
3   1 days 00:00:08
dtype: timedelta64[ns]

In [219]:
s.dt.days

0    1
1    1
2    1
3    1
dtype: int64

In [220]:
s.dt.seconds

0    5
1    6
2    7
3    8
dtype: int64

In [221]:
s.dt.components

,days,hours,minutes,seconds,milliseconds,microseconds,nanoseconds
0,1,0,0,5,0,0,0
1,1,0,0,6,0,0,0
2,1,0,0,7,0,0,0
3,1,0,0,8,0,0,0


>Note
>
>Series.dt will raise a TypeError if you access with a non-datetime-like values.

## Vectorized string methods
Series is equipped with a set of string processing methods that make it easy to operate on each element of the array. Perhaps most importantly, these methods exclude missing/NA values automatically. These are accessed via the Series’s str attribute and generally have names matching the equivalent (scalar) built-in string methods. For example:

In [222]:
s = pd.Series(
    ["A", "B", "C", "Aaba", "Baca", np.nan, "CABA", "dog", "cat"], dtype="string"
)
s.str.lower()

0       a
1       b
2       c
3    aaba
4    baca
5    <NA>
6    caba
7     dog
8     cat
dtype: string

Powerful pattern-matching methods are provided as well, but note that pattern-matching generally uses regular expressions by default (and in some cases always uses them).

>Note
>
>Prior to pandas 1.0, string methods were only available on object -dtype Series. pandas 1.0 added the StringDtype which is dedicated to strings. See Text data types for more.

Please see Vectorized String Methods for a complete description.

## Sorting
pandas supports three kinds of sorting: sorting by index labels, sorting by column values, and sorting by a combination of both.

### By index
The Series.sort_index() and DataFrame.sort_index() methods are used to sort a pandas object by its index levels.

In [223]:
df = pd.DataFrame(
    {
        "one": pd.Series(np.random.randn(3), index=["a", "b", "c"]),
        "two": pd.Series(np.random.randn(4), index=["a", "b", "c", "d"]),
        "three": pd.Series(np.random.randn(3), index=["b", "c", "d"]),
    }
)

unsorted_df = df.reindex(
    index=["a", "d", "c", "b"], columns=["three", "two", "one"]
)

unsorted_df

,three,two,one
a,NaN,0.426603,-2.141949
d,-0.741376,0.859392,NaN
c,0.157176,2.539783,0.340247
b,-0.692272,0.151815,-0.508248


In [224]:
# DataFrame
unsorted_df.sort_index()

,three,two,one
a,NaN,0.426603,-2.141949
b,-0.692272,0.151815,-0.508248
c,0.157176,2.539783,0.340247
d,-0.741376,0.859392,NaN


In [225]:
unsorted_df.sort_index(ascending=False)

,three,two,one
d,-0.741376,0.859392,NaN
c,0.157176,2.539783,0.340247
b,-0.692272,0.151815,-0.508248
a,NaN,0.426603,-2.141949


In [226]:
unsorted_df.sort_index(axis=1)

,one,three,two
a,-2.141949,NaN,0.426603
d,NaN,-0.741376,0.859392
c,0.340247,0.157176,2.539783
b,-0.508248,-0.692272,0.151815


In [227]:
# Series
unsorted_df["three"].sort_index()

a         NaN
b   -0.692272
c    0.157176
d   -0.741376
Name: three, dtype: float64

*New in version 1.1.0.*

Sorting by index also supports a key parameter that takes a callable function to apply to the index being sorted. For MultiIndex objects, the key is applied per-level to the levels specified by level.

In [228]:
s1 = pd.DataFrame({"a": ["B", "a", "C"], "b": [1, 2, 3], "c": [2, 3, 4]}).set_index(
    list("ab")
)
s1

,,c
a,b,
B,1,2
a,2,3
C,3,4


In [229]:
s1.sort_index(level="a")

,,c
a,b,
B,1,2
C,3,4
a,2,3


In [230]:
s1.sort_index(level="a", key=lambda idx: idx.str.lower())

,,c
a,b,
a,2,3
B,1,2
C,3,4


For information on key sorting by value, see value sorting.

### By values
The Series.sort_values() method is used to sort a Series by its values. The DataFrame.sort_values() method is used to sort a DataFrame by its column or row values. The optional by parameter to DataFrame.sort_values() may used to specify one or more columns to use to determine the sorted order.

In [231]:
df1 = pd.DataFrame(
    {"one": [2, 1, 1, 1], "two": [1, 3, 2, 4], "three": [5, 4, 3, 2]}
)
df1.sort_values(by="two")

,one,two,three
0,2,1,5
2,1,2,3
1,1,3,4
3,1,4,2


The by parameter can take a list of column names, e.g.:

In [232]:
df1[["one", "two", "three"]].sort_values(by=["one", "two"])

,one,two,three
2,1,2,3
1,1,3,4
3,1,4,2
0,2,1,5


These methods have special treatment of NA values via the na_position argument:

In [233]:
s[2] = np.nan
s.sort_values()

0       A
3    Aaba
1       B
4    Baca
6    CABA
8     cat
7     dog
2    <NA>
5    <NA>
dtype: string

In [234]:
s.sort_values(na_position="first")

2    <NA>
5    <NA>
0       A
3    Aaba
1       B
4    Baca
6    CABA
8     cat
7     dog
dtype: string

*New in version 1.1.0.*

Sorting also supports a key parameter that takes a callable function to apply to the values being sorted.

In [235]:
s1 = pd.Series(["B", "a", "C"])
s1.sort_values()

0    B
2    C
1    a
dtype: object

In [236]:
s1.sort_values(key=lambda x: x.str.lower())

1    a
0    B
2    C
dtype: object

key will be given the Series of values and should return a Series or array of the same shape with the transformed values. For DataFrame objects, the key is applied per column, so the key should still expect a Series and return a Series, e.g.

In [237]:
df = pd.DataFrame({"a": ["B", "a", "C"], "b": [1, 2, 3]})
df.sort_values(by="a")

,a,b
0,B,1
2,C,3
1,a,2


In [238]:
df.sort_values(by="a", key=lambda col: col.str.lower())

,a,b
1,a,2
0,B,1
2,C,3


The name or type of each column can be used to apply different functions to different columns.

### By indexes and values
Strings passed as the by parameter to DataFrame.sort_values() may refer to either columns or index level names.

In [239]:
# Build MultiIndex
idx = pd.MultiIndex.from_tuples(
    [("a", 1), ("a", 2), ("a", 2), ("b", 2), ("b", 1), ("b", 1)]
)

idx.names = ["first", "second"]

# Build DataFrame
df_multi = pd.DataFrame({"A": np.arange(6, 0, -1)}, index=idx)

df_multi

A
first second   
a     1       6
      2       5
      2       4
b     2       3
      1       2
      1       1

Sort by ‘second’ (index) and ‘A’ (column)

In [240]:
df_multi.sort_values(by=["second", "A"])

A
first second   
b     1       1
      1       2
a     1       6
b     2       3
a     2       4
      2       5

>Note
>
>If a string matches both a column name and an index level name then a warning is issued and the column takes precedence. This will result in an ambiguity error in a future version.

### searchsorted
Series has the searchsorted() method, which works similarly to numpy.ndarray.searchsorted().

In [241]:
ser = pd.Series([1, 2, 3])
ser.searchsorted([0, 3])

array([0, 2], dtype=int64)

In [242]:
ser.searchsorted([0, 4])

array([0, 3], dtype=int64)

In [243]:
ser.searchsorted([1, 3], side="right")

array([1, 3], dtype=int64)

In [244]:
ser.searchsorted([1, 3], side="left")

array([0, 2], dtype=int64)

In [245]:
ser = pd.Series([3, 1, 2])
ser.searchsorted([0, 3], sorter=np.argsort(ser))

array([0, 2], dtype=int64)

### smallest / largest values
Series has the nsmallest() and nlargest() methods which return the smallest or largest n values. For a large Series this can be much faster than sorting the entire Series and calling head(n) on the result.

In [246]:
s = pd.Series(np.random.permutation(10))
s

0    2
1    9
2    0
3    1
4    8
5    6
6    3
7    7
8    4
9    5
dtype: int32

In [247]:
s.sort_values()

2    0
3    1
0    2
6    3
8    4
9    5
5    6
7    7
4    8
1    9
dtype: int32

In [248]:
s.nsmallest(3)

2    0
3    1
0    2
dtype: int32

In [249]:
s.nlargest(3)

1    9
4    8
7    7
dtype: int32

DataFrame also has the nlargest and nsmallest methods.

In [250]:
df = pd.DataFrame(
    {
        "a": [-2, -1, 1, 10, 8, 11, -1],
        "b": list("abdceff"),
        "c": [1.0, 2.0, 4.0, 3.2, np.nan, 3.0, 4.0],
    }
)
df.nlargest(3, "a")

,a,b,c
5,11,f,3.0
3,10,c,3.2
4,8,e,NaN


In [251]:
df.nlargest(5, ["a", "c"])

,a,b,c
5,11,f,3.0
3,10,c,3.2
4,8,e,NaN
2,1,d,4.0
6,-1,f,4.0


In [252]:
df.nsmallest(3, "a")

,a,b,c
0,-2,a,1.0
1,-1,b,2.0
6,-1,f,4.0


In [253]:
df.nsmallest(5, ["a", "c"])

,a,b,c
0,-2,a,1.0
1,-1,b,2.0
6,-1,f,4.0
2,1,d,4.0
4,8,e,NaN


### Sorting by a MultiIndex column
You must be explicit about sorting when the column is a MultiIndex, and fully specify all levels to by.

In [254]:
df1.columns = pd.MultiIndex.from_tuples(
    [("a", "one"), ("a", "two"), ("b", "three")]
)
df1.sort_values(by=("a", "two"))

a         b
  one two three
0   2   1     5
2   1   2     3
1   1   3     4
3   1   4     2

## Copying
The copy() method on pandas objects copies the underlying data (though not the axis indexes, since they are immutable) and returns a new object. Note that it is seldom necessary to copy objects. For example, there are only a handful of ways to alter a DataFrame in-place:

Inserting, deleting, or modifying a column.

Assigning to the index or columns attributes.

For homogeneous data, directly modifying the values via the values attribute or advanced indexing.

To be clear, no pandas method has the side effect of modifying your data; almost every method returns a new object, leaving the original object untouched. If the data is modified, it is because you did so explicitly.

### dtypes
For the most part, pandas uses NumPy arrays and dtypes for Series or individual columns of a DataFrame. NumPy provides support for float, int, bool, timedelta64[ns] and datetime64[ns] (note that NumPy does not support timezone-aware datetimes).

pandas and third-party libraries extend NumPy’s type system in a few places. This section describes the extensions pandas has made internally. See Extension types for how to write your own extension that works with pandas. See Extension data types for a list of third-party libraries that have implemented an extension.

The following table lists all of pandas extension types. For methods requiring dtype arguments, strings can be specified as indicated. See the respective documentation sections for more on each type.

| Kind of Data        | Data Type        | Scalar           | Array                | String Aliases         | Documentation      |
|---------------------|------------------|------------------|----------------------|------------------------|--------------------|
| tz-aware datetime   | DatetimeTZDtype  | Timestamp        | arrays.DatetimeArray | 'datetime64[ns, <tz>]' | Time zone handling | 
| Categorical         | CategoricalDtype | (none)           | Categorical          | 'category'             | Categorical data   |
| period (time spans) | PeriodDtype      | Period           | arrays.PeriodArray   | 'period[<freq>]', 'Period[<freq>]' | Time span representation | 
| sparse              | SparseDtype      | (none)           | arrays.SparseArray   | 'Sparse', 'Sparse[int]', 'Sparse[float]' | Sparse data structures | 
| intervals           | IntervalDtype    | Interval         | arrays.IntervalArray | 'interval', 'Interval', 'Interval[<numpy_dtype>]', 'Interval[datetime64[ns, <tz>]]', 'Interval[timedelta64[<freq>]]' | IntervalIndex | 
| nullable integer    | Int64Dtype, …    | (none)           | arrays.IntegerArray  | 'Int8', 'Int16', 'Int32', 'Int64', 'UInt8', 'UInt16', 'UInt32', 'UInt64' | Nullable integer data type | 
| Strings             | StringDtype      | str              | arrays.StringArray   | 'string'               | Working with text data | 
| Boolean (with NA)   | BooleanDtype     | bool             | arrays.BooleanArray  | 'boolean'              | Boolean data with missing values | 

pandas has two ways to store strings.

1. object dtype, which can hold any Python object, including strings.
2. StringDtype, which is dedicated to strings.

Generally, we recommend using StringDtype. See Text data types for more.

Finally, arbitrary objects may be stored using the object dtype, but should be avoided to the extent possible (for performance and interoperability with other libraries and methods. See object conversion).

A convenient dtypes attribute for DataFrame returns a Series with the data type of each column.

In [255]:
dft = pd.DataFrame(
    {
        "A": np.random.rand(3),
        "B": 1,
        "C": "foo",
        "D": pd.Timestamp("20010102"),
        "E": pd.Series([1.0] * 3).astype("float32"),
        "F": False,
        "G": pd.Series([1] * 3, dtype="int8"),
    }
)
dft

,A,B,C,D,E,F,G
0,0.101884,1,foo,2001-01-02,1.0,False,1
1,0.515168,1,foo,2001-01-02,1.0,False,1
2,0.713523,1,foo,2001-01-02,1.0,False,1


In [256]:
dft.dtypes

A           float64
B             int64
C            object
D    datetime64[ns]
E           float32
F              bool
G              int8
dtype: object

On a Series object, use the dtype attribute.

In [257]:
dft["A"].dtype

dtype('float64')

If a pandas object contains data with multiple dtypes in a single column, the dtype of the column will be chosen to accommodate all of the data types (object is the most general).

In [258]:
# these ints are coerced to floats
pd.Series([1, 2, 3, 4, 5, 6.0])

0    1.0
1    2.0
2    3.0
3    4.0
4    5.0
5    6.0
dtype: float64

In [259]:
# string data forces an ``object`` dtype
pd.Series([1, 2, 3, 6.0, "foo"])

0      1
1      2
2      3
3    6.0
4    foo
dtype: object

The number of columns of each type in a DataFrame can be found by calling DataFrame.dtypes.value_counts().

In [260]:
dft.dtypes.value_counts()

int8              1
float32           1
bool              1
object            1
datetime64[ns]    1
float64           1
int64             1
dtype: int64

Numeric dtypes will propagate and can coexist in DataFrames. If a dtype is passed (either directly via the dtype keyword, a passed ndarray, or a passed Series), then it will be preserved in DataFrame operations. Furthermore, different numeric dtypes will NOT be combined. The following example will give you a taste.

In [261]:
df1 = pd.DataFrame(np.random.randn(8, 1), columns=["A"], dtype="float32")
df1

,A
0,-2.211995
1,0.611830
2,-2.063502
3,0.679016
4,-0.930366
5,-1.609086
6,1.498008
7,0.698789


In [262]:
df1.dtypes

A    float32
dtype: object

In [263]:
df2 = pd.DataFrame(
    {
        "A": pd.Series(np.random.randn(8), dtype="float16"),
        "B": pd.Series(np.random.randn(8)),
        "C": pd.Series(np.array(np.random.randn(8), dtype="uint8")),
    }
)
df2

,A,B,C
0,1.180664,0.746602,0
1,-0.426025,-0.367747,0
2,0.416260,1.541429,0
3,-0.135620,-1.570791,0
4,-0.324463,-0.502573,0
5,-0.199829,1.430857,255
6,-0.958008,-0.082505,0
7,2.197266,-0.192187,255


In [264]:
df2.dtypes

A    float16
B    float64
C      uint8
dtype: object

### defaults
By default integer types are int64 and float types are float64, regardless of platform (32-bit or 64-bit). The following will all result in int64 dtypes.

In [265]:
pd.DataFrame([1, 2], columns=["a"]).dtypes

a    int64
dtype: object

In [266]:
pd.DataFrame({"a": [1, 2]}).dtypes

a    int64
dtype: object

In [267]:
pd.DataFrame({"a": 1}, index=list(range(2))).dtypes

a    int64
dtype: object

Note that Numpy will choose platform-dependent types when creating arrays. The following WILL result in int32 on 32-bit platform.

In [268]:
frame = pd.DataFrame(np.array([1, 2]))

### upcasting
Types can potentially be upcasted when combined with other types, meaning they are promoted from the current type (e.g. int to float).

In [269]:
df3 = df1.reindex_like(df2).fillna(value=0.0) + df2
df3

,A,B,C
0,-1.031331,0.746602,0.0
1,0.185805,-0.367747,0.0
2,-1.647242,1.541429,0.0
3,0.543396,-1.570791,0.0
4,-1.254829,-0.502573,0.0
5,-1.808915,1.430857,255.0
6,0.540000,-0.082505,0.0
7,2.896055,-0.192187,255.0


In [270]:
df3.dtypes

A    float32
B    float64
C    float64
dtype: object

DataFrame.to_numpy() will return the lower-common-denominator of the dtypes, meaning the dtype that can accommodate ALL of the types in the resulting homogeneous dtyped NumPy array. This can force some upcasting.

In [271]:
df3.to_numpy().dtype

dtype('float64')

### astype
You can use the astype() method to explicitly convert dtypes from one to another. These will by default return a copy, even if the dtype was unchanged (pass copy=False to change this behavior). In addition, they will raise an exception if the astype operation is invalid.

Upcasting is always according to the NumPy rules. If two different dtypes are involved in an operation, then the more general one will be used as the result of the operation.

In [272]:
df3

,A,B,C
0,-1.031331,0.746602,0.0
1,0.185805,-0.367747,0.0
2,-1.647242,1.541429,0.0
3,0.543396,-1.570791,0.0
4,-1.254829,-0.502573,0.0
5,-1.808915,1.430857,255.0
6,0.540000,-0.082505,0.0
7,2.896055,-0.192187,255.0


In [273]:
df3.dtypes

A    float32
B    float64
C    float64
dtype: object

In [274]:
# conversion of dtypes
df3.astype("float32").dtypes

A    float32
B    float32
C    float32
dtype: object

Convert a subset of columns to a specified type using astype().

In [275]:
dft = pd.DataFrame({"a": [1, 2, 3], "b": [4, 5, 6], "c": [7, 8, 9]})
dft[["a", "b"]] = dft[["a", "b"]].astype(np.uint8)
dft

,a,b,c
0,1,4,7
1,2,5,8
2,3,6,9


In [276]:
dft.dtypes

a    uint8
b    uint8
c    int64
dtype: object

Convert certain columns to a specific dtype by passing a dict to astype().

In [277]:
dft1 = pd.DataFrame({"a": [1, 0, 1], "b": [4, 5, 6], "c": [7, 8, 9]})
dft1 = dft1.astype({"a": np.bool_, "c": np.float64})
dft1

,a,b,c
0,True,4,7.0
1,False,5,8.0
2,True,6,9.0


In [278]:
dft1.dtypes

a       bool
b      int64
c    float64
dtype: object

>Note
>
>When trying to convert a subset of columns to a specified type using astype() and loc(), upcasting occurs.

loc() tries to fit in what we are assigning to the current dtypes, while [] will overwrite them taking the dtype from the right hand side. Therefore the following piece of code produces the unintended result.

In [279]:
dft = pd.DataFrame({"a": [1, 2, 3], "b": [4, 5, 6], "c": [7, 8, 9]})
dft.loc[:, ["a", "b"]].astype(np.uint8).dtypes

a    uint8
b    uint8
dtype: object

In [280]:
dft.loc[:, ["a", "b"]] = dft.loc[:, ["a", "b"]].astype(np.uint8)
dft.dtypes

a    int64
b    int64
c    int64
dtype: object

### object conversion
pandas offers various functions to try to force conversion of types from the object dtype to other types. In cases where the data is already of the correct type, but stored in an object array, the DataFrame.infer_objects() and Series.infer_objects() methods can be used to soft convert to the correct type.

In [281]:
import datetime

df = pd.DataFrame(
    [
        [1, 2],
        ["a", "b"],
        [datetime.datetime(2016, 3, 2), datetime.datetime(2016, 3, 2)],
    ]
)
df = df.T
df

,0,1,2
0,1,a,2016-03-02
1,2,b,2016-03-02


In [282]:
df.dtypes

0            object
1            object
2    datetime64[ns]
dtype: object

Because the data was transposed the original inference stored all columns as object, which infer_objects will correct.

In [283]:
df.infer_objects().dtypes

0             int64
1            object
2    datetime64[ns]
dtype: object

The following functions are available for one dimensional object arrays or scalars to perform hard conversion of objects to a specified type:

* to_numeric() (conversion to numeric dtypes)

In [284]:
m = ["1.1", 2, 3]
pd.to_numeric(m)

array([1.1, 2. , 3. ])

* to_datetime() (conversion to datetime objects)

In [285]:
import datetime
m = ["2016-07-09", datetime.datetime(2016, 3, 2)]
pd.to_datetime(m)

DatetimeIndex(['2016-07-09', '2016-03-02'], dtype='datetime64[ns]', freq=None)

* to_timedelta() (conversion to timedelta objects)

In [286]:
m = ["5us", pd.Timedelta("1day")]
pd.to_timedelta(m)

TimedeltaIndex(['0 days 00:00:00.000005', '1 days 00:00:00'], dtype='timedelta64[ns]', freq=None)

To force a conversion, we can pass in an errors argument, which specifies how pandas should deal with elements that cannot be converted to desired dtype or object. By default, errors='raise', meaning that any errors encountered will be raised during the conversion process. However, if errors='coerce', these errors will be ignored and pandas will convert problematic elements to pd.NaT (for datetime and timedelta) or np.nan (for numeric). This might be useful if you are reading in data which is mostly of the desired dtype (e.g. numeric, datetime), but occasionally has non-conforming elements intermixed that you want to represent as missing:

In [287]:
import datetime
m = ["apple", datetime.datetime(2016, 3, 2)]
pd.to_datetime(m, errors="coerce")

DatetimeIndex(['NaT', '2016-03-02'], dtype='datetime64[ns]', freq=None)

In [288]:
m = ["apple", 2, 3]
pd.to_numeric(m, errors="coerce")

array([nan,  2.,  3.])

In [289]:
m = ["apple", pd.Timedelta("1day")]
pd.to_timedelta(m, errors="coerce")

TimedeltaIndex([NaT, '1 days'], dtype='timedelta64[ns]', freq=None)

The errors parameter has a third option of errors='ignore', which will simply return the passed in data if it encounters any errors with the conversion to a desired data type:

In [290]:
import datetime
m = ["apple", datetime.datetime(2016, 3, 2)]
pd.to_datetime(m, errors="ignore")

Index(['apple', 2016-03-02 00:00:00], dtype='object')

In [291]:
m = ["apple", 2, 3]
pd.to_numeric(m, errors="ignore")

array(['apple', 2, 3], dtype=object)

In [292]:
m = ["apple", pd.Timedelta("1day")]
pd.to_timedelta(m, errors="ignore")

array(['apple', Timedelta('1 days 00:00:00')], dtype=object)

In addition to object conversion, to_numeric() provides another argument downcast, which gives the option of downcasting the newly (or already) numeric data to a smaller dtype, which can conserve memory:

In [293]:
m = ["1", 2, 3]
pd.to_numeric(m, downcast="integer")  # smallest signed int dtype

array([1, 2, 3], dtype=int8)

In [294]:
pd.to_numeric(m, downcast="signed")  # same as 'integer'

array([1, 2, 3], dtype=int8)

In [295]:
pd.to_numeric(m, downcast="unsigned")  # smallest unsigned int dtype

array([1, 2, 3], dtype=uint8)

In [296]:
pd.to_numeric(m, downcast="float")  # smallest float dtype

array([1., 2., 3.], dtype=float32)

As these methods apply only to one-dimensional arrays, lists or scalars; they cannot be used directly on multi-dimensional objects such as DataFrames. However, with apply(), we can “apply” the function over each column efficiently:

In [297]:
import datetime
df = pd.DataFrame([["2016-07-09", datetime.datetime(2016, 3, 2)]] * 2, dtype="O")
df

,0,1
0,2016-07-09,2016-03-02 00:00:00
1,2016-07-09,2016-03-02 00:00:00


In [298]:
df.apply(pd.to_datetime)

,0,1
0,2016-07-09,2016-03-02
1,2016-07-09,2016-03-02


In [299]:
df = pd.DataFrame([["1.1", 2, 3]] * 2, dtype="O")
df

,0,1,2
0,1.1,2,3
1,1.1,2,3


In [300]:
df.apply(pd.to_numeric)

,0,1,2
0,1.1,2,3
1,1.1,2,3


In [301]:
df = pd.DataFrame([["5us", pd.Timedelta("1day")]] * 2, dtype="O")
df

,0,1
0,5us,1 days 00:00:00
1,5us,1 days 00:00:00


In [302]:
df.apply(pd.to_timedelta)

,0,1
0,0 days 00:00:00.000005,1 days
1,0 days 00:00:00.000005,1 days


### gotchas
Performing selection operations on integer type data can easily upcast the data to floating. The dtype of the input data will be preserved in cases where nans are not introduced. See also Support for integer NA.

In [303]:
dfi = df3.astype("int32")
dfi["E"] = 1
dfi

,A,B,C,E
0,-1,0,0,1
1,0,0,0,1
2,-1,1,0,1
3,0,-1,0,1
4,-1,0,0,1
5,-1,1,255,1
6,0,0,0,1
7,2,0,255,1


In [304]:
dfi.dtypes

A    int32
B    int32
C    int32
E    int64
dtype: object

In [305]:
casted = dfi[dfi > 0]
casted

,A,B,C,E
0,NaN,NaN,NaN,1
1,NaN,NaN,NaN,1
2,NaN,1.0,NaN,1
3,NaN,NaN,NaN,1
4,NaN,NaN,NaN,1
5,NaN,1.0,255.0,1
6,NaN,NaN,NaN,1
7,2.0,NaN,255.0,1


In [306]:
casted.dtypes

A    float64
B    float64
C    float64
E      int64
dtype: object

While float dtypes are unchanged.

In [307]:
dfa = df3.copy()
dfa["A"] = dfa["A"].astype("float32")
dfa.dtypes

A    float32
B    float64
C    float64
dtype: object

In [308]:
casted = dfa[df2 > 0]
casted

,A,B,C
0,-1.031331,0.746602,NaN
1,NaN,NaN,NaN
2,-1.647242,1.541429,NaN
3,NaN,NaN,NaN
4,NaN,NaN,NaN
5,NaN,1.430857,255.0
6,NaN,NaN,NaN
7,2.896055,NaN,255.0


In [309]:
casted.dtypes

A    float32
B    float64
C    float64
dtype: object

## Selecting columns based on dtype
The select_dtypes() method implements subsetting of columns based on their dtype.

First, let’s create a DataFrame with a slew of different dtypes:

In [310]:
df = pd.DataFrame(
    {
        "string": list("abc"),
        "int64": list(range(1, 4)),
        "uint8": np.arange(3, 6).astype("u1"),
        "float64": np.arange(4.0, 7.0),
        "bool1": [True, False, True],
        "bool2": [False, True, False],
        "dates": pd.date_range("now", periods=3),
        "category": pd.Series(list("ABC")).astype("category"),
    }
)

df["tdeltas"] = df.dates.diff()
df["uint64"] = np.arange(3, 6).astype("u8")
df["other_dates"] = pd.date_range("20130101", periods=3)
df["tz_aware_dates"] = pd.date_range("20130101", periods=3, tz="US/Eastern")
df

,string,int64,uint8,float64,bool1,bool2,dates,category,tdeltas,uint64,other_dates,tz_aware_dates
0,a,1,3,4.0,True,False,2021-06-03 22:07:56.749050,A,NaT,3,2013-01-01,2013-01-01 00:00:00-05:00
1,b,2,4,5.0,False,True,2021-06-04 22:07:56.749050,B,1 days,4,2013-01-02,2013-01-02 00:00:00-05:00
2,c,3,5,6.0,True,False,2021-06-05 22:07:56.749050,C,1 days,5,2013-01-03,2013-01-03 00:00:00-05:00


And the dtypes:

In [311]:
df.dtypes

string                                object
int64                                  int64
uint8                                  uint8
float64                              float64
bool1                                   bool
bool2                                   bool
dates                         datetime64[ns]
category                            category
tdeltas                      timedelta64[ns]
uint64                                uint64
other_dates                   datetime64[ns]
tz_aware_dates    datetime64[ns, US/Eastern]
dtype: object

select_dtypes() has two parameters include and exclude that allow you to say “give me the columns with these dtypes” (include) and/or “give the columns without these dtypes” (exclude).

For example, to select bool columns:

In [312]:
df.select_dtypes(include=[bool])

,bool1,bool2
0,True,False
1,False,True
2,True,False


You can also pass the name of a dtype in the NumPy dtype hierarchy:

In [313]:
df.select_dtypes(include=["bool"])

,bool1,bool2
0,True,False
1,False,True
2,True,False


select_dtypes() also works with generic dtypes as well.

For example, to select all numeric and boolean columns while excluding unsigned integers:

In [314]:
df.select_dtypes(include=["number", "bool"], exclude=["unsignedinteger"])

,int64,float64,bool1,bool2,tdeltas
0,1,4.0,True,False,NaT
1,2,5.0,False,True,1 days
2,3,6.0,True,False,1 days


To select string columns you must use the object dtype:

In [315]:
df.select_dtypes(include=["object"])

,string
0,a
1,b
2,c


To see all the child dtypes of a generic dtype like numpy.number you can define a function that returns a tree of child dtypes:

In [316]:
def subdtypes(dtype):
    subs = dtype.__subclasses__()
    if not subs:
        return dtype
    return [dtype, [subdtypes(dt) for dt in subs]]

All NumPy dtypes are subclasses of numpy.generic:

In [317]:
subdtypes(np.generic)

[numpy.generic,
 [[numpy.number,
   [[numpy.integer,
     [[numpy.signedinteger,
       [numpy.int8,
        numpy.int16,
        numpy.intc,
        numpy.int32,
        numpy.int64,
        numpy.timedelta64]],
      [numpy.unsignedinteger,
       [numpy.uint8, numpy.uint16, numpy.uintc, numpy.uint32, numpy.uint64]]]],
    [numpy.inexact,
     [[numpy.floating,
       [numpy.float16, numpy.float32, numpy.float64, numpy.longdouble]],
      [numpy.complexfloating,
       [numpy.complex64, numpy.complex128, numpy.clongdouble]]]]]],
  [numpy.flexible,
   [[numpy.character, [numpy.bytes_, numpy.str_]],
    [numpy.void, [numpy.record]]]],
  numpy.bool_,
  numpy.datetime64,
  numpy.object_]]

>Note
>
>pandas also defines the types category, and datetime64[ns, tz], which are not integrated into the normal NumPy hierarchy and won’t show up with the above function.